# Synthetic Data Generation & Fine-Tuning (QLoRA) Assignment

## Project Design

This week’s project involves creating a synthetic dataset and using it to fine-tune the model for better academic Q\&A performance. The plan is as follows:

1. **Data Sampling:** Select **100 academic papers** (e.g. using their abstracts and key sections). Ensure a diverse mix of subjects or paper types to provide a broad training base.
2. **Synthetic Q\&A Generation:** Use **GPT-4** to generate \~5 question-answer pairs for each paper. Craft a prompt that provides GPT-4 with the paper’s abstract or content and asks for informative Q\&A pairs. The questions should cover important points, definitions, or insights from the paper, and the answers should be correct summaries or explanations based on the text. This yields roughly **500 Q\&A pairs** in total.
3. **Include Edge-Case Examples:** Incorporate some **edge-case questions** among the above pairs – for example, a question that reflects a misunderstanding or a **hallucinated detail** about the paper. For these, provide an answer that corrects the false premise or clarifies that the paper doesn’t contain that information. Including a few such Q\&A examples (e.g. *“Q: According to the paper, what is the value of constant XYZ?”* when XYZ is not actually in the paper, and *“A: The paper does not specify XYZ; in fact, that detail is not discussed.”*) will teach the model to handle incorrect or unanswerable queries gracefully.
4. **Format Data for Instruction Tuning:** Convert all the Q\&A pairs into the **instruction-tuning JSONL format** expected by our fine-tuning pipeline. Each line in the dataset should represent a complete prompt-response dialogue. We will use a chat-style format with explicit roles. For example, you can prepend a fixed system instruction (such as `"You are a helpful academic assistant."`) and then format each Q\&A as:

   ```
   <|system|> You are a helpful academic Q&A assistant specialized in scholarly content.
   <|user|> [Question from the dataset]
   <|assistant|> [Answer from the dataset]
   ```

   Structure each JSONL entry to contain this composite prompt. This ensures the model is trained in a conversational format where it receives a user question and produces an answer, following any system instructions (tone, style) you provided.
5. **Fine-Tune LLaMA 3 7B with QLoRA:** Run a fine-tuning job on **Google Colab** (or a local GPU) using **QLoRA** via the Unsloth library. QLoRA (Quantized LoRA) will load the 7B model in 4-bit precision and train low-rank adaptation weights. This drastically lowers memory usage, allowing even a 7B (and larger) model to be fine-tuned on a single GPU without out-of-memory errors. Using Unsloth’s tools, load the base LLaMA 3 (7B) model (preferably an instruct variant) and fine-tune it on your synthetic Q\&A dataset. We’ll use LoRA adapters so the base model weights remain fixed; the training will produce a small set of adapted weights after 1–3 epochs over the dataset. *(Expect the fine-tuning to be relatively fast given \~500 examples — on a T4 or similar GPU, a few epochs should only take minutes.)*
6. **Evaluation (Pre vs. Post-Tuning):** Finally, evaluate the model’s academic QA performance **before and after fine-tuning**. Prepare a set of **10 test questions** covering various papers or concepts (you can come up with these manually, ensuring they are challenging). Run the original base model and the fine-tuned model on each question, and compare the answers. Look for improvements such as: the fine-tuned model’s answers are more detailed, use terminology from the papers, correct mistakes the base model made, or cite relevant concepts from the training data. This comparison will let you quantify accuracy gains. You might measure accuracy as the number of questions answered correctly or with relevant info, or simply note qualitatively how the responses differ.

Throughout this design, the key idea is that **domain-aligned data** will make the model more knowledgeable in that domain. Instead of the agent relying solely on retrieval each time, the fine-tuned model will have *internalized* some academic knowledge and answer patterns. Fine-tuning on a well-structured QA dataset (as opposed to just dumping raw text) is crucial for the model to learn effectively.

## Setup

In [1]:
pip install arxiv openai tqdm pandas jsonlines dotenv pypdf


Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 33.5 MB/s  0:00:00
  DEPRECATION: Building 'sgmllib3k' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'sgmllib3k'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6089 sha256=bf375e0b978f7b18739692d5bec43fb631668063aaab1de3e387f31e99232845
  Stored in directory: /tmp/pip-ephem-wheel-cache-va_s13tz/wheels/f0/69/93/a47e9d621be168e9e33c7ce60524393c0b92ae83cf

In [13]:
# Imports
import arxiv
from openai import OpenAI
import json
import jsonlines
import pandas as pd
from tqdm import tqdm
import random
from dotenv import load_dotenv
import os
import re
import json
import time

# 🔑 Set your OpenAI API key

# Load environment variables (make sure you have OPENAI_API_KEY in your .env file)
from pathlib import Path
from dotenv import load_dotenv

ENV_PATH = Path.cwd() / ".env"
load_dotenv(dotenv_path=ENV_PATH, override=True)

print("Using .env at:", ENV_PATH)


# Initialize client
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Create output directory
os.makedirs("dataset_arxiv_json", exist_ok=True)

Using .env at: /home/jovyan/work/MLE_in_Gen_AI-Course/class7/Homework_Kevin/.env


## Data Sampling

In [ ]:
import os, re, json, gzip, time, hashlib, tempfile
from pathlib import Path
from typing import Optional, Dict, Any, Iterable, Set

import arxiv
from tqdm import tqdm
from pypdf import PdfReader

# -----------------------------
# Config
# -----------------------------
CATEGORIES = [
    "q-fin.MF", "q-fin.TR", "q-fin.ST", "q-fin.RM", "q-fin.PR",
    "q-fin.CP", "q-fin", "q-fin.EC", "q-fin.GN", "q-fin.PM", 
    "econ.EM", "econ.GN", "econ.TH"
]
MAX_RESULTS_PER_CAT = 50

FIRST_N_PAGES = 10
MAX_BODY_CHARS = 25000
MAX_ABS_CHARS  = 3000
MIN_BODY_CHARS = 3000


OUT_DIR = Path("arxiv_finance_corpus")
OUT_DIR.mkdir(parents=True, exist_ok=True)

META_PATH = OUT_DIR / "papers_meta.jsonl.gz"
TEXT_PATH = OUT_DIR / "papers_text_p1_10.jsonl.gz"

SLEEP_BETWEEN_REQUESTS_SEC = 0.5  # be polite / avoid throttling

# -----------------------------
# Utilities
# -----------------------------
_ws = re.compile(r"\s+")
def clean_text(s: str) -> str:
    if not s:
        return ""
    s = s.replace("\x00", " ")
    s = _ws.sub(" ", s).strip()
    return s

def safe_trunc(s: str, max_chars: int) -> str:
    if len(s) <= max_chars:
        return s
    # cut on a boundary to avoid mid-word spam
    cut = s[:max_chars]
    last_space = cut.rfind(" ")
    return (cut[:last_space] if last_space > 0 else cut).rstrip()

def sha1(s: str) -> str:
    return hashlib.sha1(s.encode("utf-8", errors="ignore")).hexdigest()

def gzip_jsonl_append(path: Path, record: Dict[str, Any]) -> None:
    # append in text mode
    with gzip.open(path, "at", encoding="utf-8") as f:
        f.write(json.dumps(record, ensure_ascii=False) + "\n")

def extract_first_pages_text(pdf_path: Path, n_pages: int) -> str:
    reader = PdfReader(str(pdf_path))
    texts = []
    for i in range(min(n_pages, len(reader.pages))):
        try:
            t = reader.pages[i].extract_text() or ""
        except Exception:
            t = ""
        texts.append(t)
    return "\n".join(texts)

def download_pdf_to_temp(result: arxiv.Result) -> Path:
    # arxiv library can download via result.download_pdf()
    # We force temp dir and then delete after.
    tmpdir = Path(tempfile.mkdtemp(prefix="arxiv_pdf_"))
    pdf_path = Path(result.download_pdf(dirpath=str(tmpdir)))
    return pdf_path

def sizeof_dir(path: Path) -> int:
    total = 0
    for p in path.rglob("*"):
        if p.is_file():
            total += p.stat().st_size
    return total

# -----------------------------
# Main collection
# -----------------------------
seen_ids: Set[str] = set()
retrieved = 0
enriched = 0
failed_extract = 0

# fresh files each run (comment out if you want to append)
for p in [META_PATH, TEXT_PATH]:
    if p.exists():
        p.unlink()

for cat in tqdm(CATEGORIES, desc="Fetching arXiv categories"):
    search = arxiv.Search(
        query=f"cat:{cat}",
        max_results=MAX_RESULTS_PER_CAT,
        sort_by=arxiv.SortCriterion.SubmittedDate
    )

    for result in search.results():
        arxiv_id = (result.get_short_id() if hasattr(result, "get_short_id") else None) or result.entry_id
        arxiv_id = arxiv_id.replace("http://arxiv.org/abs/", "").replace("https://arxiv.org/abs/", "")
        if arxiv_id in seen_ids:
            continue
        seen_ids.add(arxiv_id)

        title = clean_text(result.title or "")
        abstract = clean_text(result.summary or "")
        abstract = safe_trunc(abstract, MAX_ABS_CHARS)

        authors = []
        try:
            authors = [a.name for a in (result.authors or [])][:8]
        except Exception:
            authors = []

        categories = []
        try:
            categories = list(getattr(result, "categories", []) or [])
        except Exception:
            categories = []

        published = None
        try:
            published = result.published.isoformat() if result.published else None
        except Exception:
            published = None

        pdf_url = ""
        try:
            pdf_url = result.pdf_url or ""
        except Exception:
            pdf_url = ""

        meta = {
            "id": arxiv_id,
            "source_category": cat,
            "primary_title": title,
            "authors": authors,
            "categories": categories,
            "published": published,
            "pdf_url": pdf_url,
            "abstract": abstract,
            "abstract_chars": len(abstract),
            "title_hash": sha1(title.lower()),
        }
        gzip_jsonl_append(META_PATH, meta)
        retrieved += 1

        # Enrich: first 5 pages text (then discard pdf)
        body_text = ""
        ok = True
        try:
            time.sleep(SLEEP_BETWEEN_REQUESTS_SEC)
            pdf_path = download_pdf_to_temp(result)
            raw = extract_first_pages_text(pdf_path, FIRST_N_PAGES)
            body_text = clean_text(raw)
            body_text = safe_trunc(body_text, MAX_BODY_CHARS)
        except Exception:
            ok = False
            failed_extract += 1
        finally:
            # delete temp dir
            try:
                if "pdf_path" in locals() and pdf_path.exists():
                    tmp = pdf_path.parent
                    for fp in tmp.glob("*"):
                        fp.unlink(missing_ok=True)
                    tmp.rmdir()
            except Exception:
                pass

        text_rec = {
            **meta,
            "body_p1_5": body_text,
            "body_chars": len(body_text),
            "extraction_ok": ok,
        }
        gzip_jsonl_append(TEXT_PATH, text_rec)
        enriched += 1

print(f"✅ Retrieved (metadata): {retrieved}")
print(f"✅ Enriched (p1-5 text): {enriched}")
print(f"⚠️ Extraction failed: {failed_extract}")

total_bytes = sizeof_dir(OUT_DIR)
print(f"📦 Output folder size: {total_bytes/1024/1024:.2f} MB")
print(f"- {META_PATH.name}: {META_PATH.stat().st_size/1024/1024:.2f} MB")
print(f"- {TEXT_PATH.name}: {TEXT_PATH.stat().st_size/1024/1024:.2f} MB")


Fetching arXiv categories:   0%|          | 0/13 [00:00<?, ?it/s]/tmp/ipykernel_4929/1861694540.py:105: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  for result in search.results():
Fetching arXiv categories:  15%|█▌        | 2/13 [02:56<16:01, 87.40s/it]Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 14 0 (offset 0)
Ignoring wrong pointing object 16 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)
Ignoring wrong pointing object 27 0 (offset 0)
Ignoring wrong pointing object 29 0 (offset 0)
Ignoring wrong pointing object 42 0 (offset 0)
Ignoring wrong pointing object 44 0 (offset 0)
Ignoring wrong pointing object 53 0 (offset 0)
Ignoring wrong pointing object 65 0 (offset 0)
Ignoring wrong pointing object 67 0 (offset 0)
Ignoring wrong pointing object 69 0 (offset 0)
Ignoring wrong pointing object 71 0 (offs

✅ Retrieved (metadata): 455
✅ Enriched (p1-5 text): 455
⚠️ Extraction failed: 3
📦 Output folder size: 4.26 MB
- papers_meta.jsonl.gz: 0.40 MB
- papers_text_p1_5.jsonl.gz: 3.86 MB


## Synthetic Data Generation

In [ ]:
import os, json, gzip, re, time, sys
from pathlib import Path
from typing import List, Dict, Optional
from openai import OpenAI
from tqdm import tqdm

# -------------------------
# Paths
# -------------------------
CORPUS_DIR = Path("arxiv_finance_corpus")
IN_PATH = CORPUS_DIR / "papers_text_p1_10.jsonl.gz"
OUT_PATH = CORPUS_DIR / "synthetic_finance_qa.chatml.jsonl"

# -------------------------
# Controls
# -------------------------
MODEL = "gpt-5"
TEMPERATURE = 1
RETRIES = 3
BACKOFF = 2
SLEEP_BETWEEN_CALLS = 0.2

MIN_BODY_CHARS = 3000
MAX_CONTEXT_CHARS = 22000

SYSTEM_PROMPT = (
    "You are a finance and investing expert. You explain concepts clearly, "
    "use correct financial terminology, and avoid speculation. "
    "When answering, you only use information provided in my training knowledge."
)

PRINT_EVERY = 10          # per N papers processed, print status
SNIPPET_CHARS = 400       # show small snippet of model output on failure
FLUSH_EVERY = 20          # flush output file every N successful papers

# -------------------------
# Helpers
# -------------------------
_ws = re.compile(r"\s+")
def clean_text(s: str) -> str:
    s = (s or "").replace("\x00", " ")
    return _ws.sub(" ", s).strip()

def trunc(s: str, max_chars: int) -> str:
    s = s or ""
    if len(s) <= max_chars:
        return s
    cut = s[:max_chars]
    last_space = cut.rfind(" ")
    return (cut[:last_space] if last_space > 0 else cut).rstrip()

def make_generation_prompt(title: str, abstract: str, body: str) -> str:
    source = f"TITLE: {title}\n\nABSTRACT: {abstract}\n\nEXCERPT: {body}"
    source = trunc(source, MAX_CONTEXT_CHARS)
    return f"""
You will create training Q&A pairs to teach a model finance/investing expertise.

Rules:
- Use ONLY my training knowledge. Do not invent facts, datasets, results, or numbers.
- If details are missing, say so explicitly (e.g., "The abstract does not specify ...").
- Questions should be what a serious finance student or analyst would ask:
  mix of conceptual, methodology, implications, limitations, and definitions.
- Answers should be precise, finance-literate, and grounded.
- Return STRICT JSON with exactly 5 items.

Output JSON schema:
{{
  "qas": [
    {{"question": "...", "answer": "..."}},
    {{"question": "...", "answer": "..."}},
    {{"question": "...", "answer": "..."}},
    {{"question": "...", "answer": "..."}},
    {{"question": "...", "answer": "..."}}
  ]
}}

SOURCE:
\"\"\"{source}\"\"\"
""".strip()

def parse_qas(content: str) -> Optional[List[Dict[str, str]]]:
    try:
        obj = json.loads(content)
    except Exception:
        return None
    if not isinstance(obj, dict) or "qas" not in obj or not isinstance(obj["qas"], list):
        return None
    qas = []
    for item in obj["qas"]:
        if not isinstance(item, dict):
            continue
        q, a = item.get("question"), item.get("answer")
        if isinstance(q, str) and isinstance(a, str):
            qas.append({"question": q.strip(), "answer": a.strip()})
    return qas if len(qas) == 5 else None

def chatml_line(system_prompt: str, user_q: str, assistant_a: str) -> Dict[str, str]:
    return {"text": f"<|system|>{system_prompt}<|user|>{user_q}<|assistant|>{assistant_a}"}

# -------------------------
# OpenAI client
# -------------------------
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
if not os.getenv("OPENAI_API_KEY"):
    raise RuntimeError("Missing OPENAI_API_KEY in environment. Add it to .env or export it.")

# -------------------------
# Load count for tqdm
# -------------------------
def count_lines_gz(path: Path) -> int:
    n = 0
    with gzip.open(path, "rt", encoding="utf-8") as f:
        for _ in f:
            n += 1
    return n

print(f"IN_PATH: {IN_PATH} (exists={IN_PATH.exists()})")
print(f"OUT_PATH: {OUT_PATH}")
print("Counting papers (for progress bar)...")
total_lines = count_lines_gz(IN_PATH)
print(f"Total records in corpus file: {total_lines}\n")

# -------------------------
# Main
# -------------------------
OUT_PATH.parent.mkdir(parents=True, exist_ok=True)

total_papers = 0
used_papers = 0
total_qas = 0

skipped_extraction = 0
skipped_short = 0
skipped_empty_body_field = 0
model_failures = 0

start_time = time.time()

with gzip.open(IN_PATH, "rt", encoding="utf-8") as fin, open(OUT_PATH, "a", encoding="utf-8") as fout:
    for line in tqdm(fin, total=total_lines, desc="Generating QAs"):
        total_papers += 1
        rec = json.loads(line)

        # Check which body field exists
        # Your corpus writer used "body_p1_5" even when FIRST_N_PAGES=10.
        # We'll try both keys and print if missing.
        body = rec.get("body_p1_10")
        if body is None:
            body = rec.get("body_p1_5", "")

        if not rec.get("extraction_ok", True):
            skipped_extraction += 1
            continue

        if int(rec.get("body_chars", 0)) < MIN_BODY_CHARS:
            skipped_short += 1
            continue

        body = clean_text(body)
        if not body:
            skipped_empty_body_field += 1
            continue

        title = clean_text(rec.get("primary_title", ""))
        abstract = clean_text(rec.get("abstract", ""))

        prompt = make_generation_prompt(title, abstract, body)

        qas = None
        last_raw = None

        for attempt in range(1, RETRIES + 1):
            try:
                t0 = time.time()
                resp = client.chat.completions.create(
                    model=MODEL,
                    messages=[{"role": "user", "content": prompt}],
                    temperature=TEMPERATURE,
                    response_format={"type": "json_object"},
                )
                dt = time.time() - t0
                last_raw = resp.choices[0].message.content
                qas = parse_qas(last_raw)
                if qas is None:
                    raise ValueError("Invalid JSON or not exactly 5 qas")
                # success
                break
            except Exception as e:
                if attempt < RETRIES:
                    print(f"\n⚠️ Paper #{total_papers} attempt {attempt} failed: {e}")
                    if last_raw:
                        snip = clean_text(last_raw)[:SNIPPET_CHARS]
                        print(f"   model snippet: {snip!r}")
                    time.sleep(BACKOFF * attempt)
                else:
                    model_failures += 1
                    print(f"\n❌ Paper #{total_papers} FAILED after {RETRIES} retries. id={rec.get('id')}")
                    print(f"   title={title[:120]!r}")
                    if last_raw:
                        snip = clean_text(last_raw)[:SNIPPET_CHARS]
                        print(f"   last model snippet: {snip!r}")
                    qas = None

        if qas is None:
            continue

        # write 5 lines
        for qa in qas:
            row = chatml_line(SYSTEM_PROMPT, qa["question"], qa["answer"])
            fout.write(json.dumps(row, ensure_ascii=False) + "\n")
            total_qas += 1

        used_papers += 1

        if used_papers % FLUSH_EVERY == 0:
            fout.flush()

        if total_papers % PRINT_EVERY == 0:
            elapsed = time.time() - start_time
            print(
                f"\n📌 Progress: read={total_papers}/{total_lines} | used={used_papers} | qas={total_qas} | "
                f"skipped(extraction={skipped_extraction}, short={skipped_short}, empty_body={skipped_empty_body_field}) | "
                f"model_failures={model_failures} | elapsed={elapsed/60:.1f}m"
            )

        time.sleep(SLEEP_BETWEEN_CALLS)

print("\n✅ DONE")
print(f"Read papers: {total_papers}")
print(f"Used papers: {used_papers}")
print(f"Total QAs written: {total_qas}")
print(f"Skipped - extraction_ok false: {skipped_extraction}")
print(f"Skipped - body_chars too small: {skipped_short}")
print(f"Skipped - empty body field: {skipped_empty_body_field}")
print(f"Model failures: {model_failures}")
print(f"Output: {OUT_PATH}")


IN_PATH: arxiv_finance_corpus/papers_text_p1_10.jsonl.gz (exists=True)
OUT_PATH: arxiv_finance_corpus/synthetic_finance_qa.chatml.jsonl
Counting papers (for progress bar)...
Total records in corpus file: 451



Generating QAs:   2%|▏         | 10/451 [07:11<4:51:18, 39.63s/it]


📌 Progress: read=10/451 | used=10 | qas=50 | skipped(extraction=0, short=0, empty_body=0) | model_failures=0 | elapsed=7.2m


Generating QAs:   4%|▍         | 19/451 [15:31<6:31:00, 54.31s/it]


📌 Progress: read=20/451 | used=20 | qas=100 | skipped(extraction=0, short=0, empty_body=0) | model_failures=0 | elapsed=17.0m


...

Generating QAs:  44%|████▍     | 199/451 [1:39:48<44:36, 10.62s/it]


❌ Paper #199 FAILED after 3 retries. id=2510.10343v2
   title='Learning the Exact SABR Model'


Generating QAs:  44%|████▍     | 199/451 [1:39:50<2:06:25, 30.10s/it]


⚠️ Paper #200 attempt 1 failed: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}


KeyboardInterrupt: 

## Ran out of Chatgpt API Balance. The below is an addition

In [ ]:
# followup_generate_qas_from_paper92.py
# Continues generation starting at paper index 92 (1-based),
# generates 2 QA pairs per paper, and APPENDS to OUT_PATH (no overwrite).

import os, json, gzip, re, time
from pathlib import Path
from typing import List, Dict, Optional
from openai import OpenAI
from tqdm import tqdm

# -------------------------
# Paths
# -------------------------
CORPUS_DIR = Path("arxiv_finance_corpus")
IN_PATH = CORPUS_DIR / "papers_text_p1_10.jsonl.gz"
OUT_PATH = CORPUS_DIR / "synthetic_finance_qa.chatml.jsonl"

# -------------------------
# Controls
# -------------------------
MODEL = "gpt-5"
TEMPERATURE = 1
RETRIES = 3
BACKOFF = 2
SLEEP_BETWEEN_CALLS = 0.2

MIN_BODY_CHARS = 3000
MAX_CONTEXT_CHARS = 22000

# Continue from paper 92 (1-based). If you want to start at paper #92 exactly,
# set START_PAPER_NUM = 92. If you want to start AFTER paper 91, also 92.
START_PAPER_NUM = 92  # 1-based inclusive

# We generate exactly 2 QAs per paper now.
QAS_PER_PAPER = 2

SYSTEM_PROMPT = (
    "You are a finance and investing expert. You explain concepts clearly, "
    "use correct financial terminology, and avoid speculation. "
    "When answering, you only use information provided in my training knowledge."
)

PRINT_EVERY = 10
SNIPPET_CHARS = 400
FLUSH_EVERY = 20

# -------------------------
# Helpers
# -------------------------
_ws = re.compile(r"\s+")
def clean_text(s: str) -> str:
    s = (s or "").replace("\x00", " ")
    return _ws.sub(" ", s).strip()

def trunc(s: str, max_chars: int) -> str:
    s = s or ""
    if len(s) <= max_chars:
        return s
    cut = s[:max_chars]
    last_space = cut.rfind(" ")
    return (cut[:last_space] if last_space > 0 else cut).rstrip()

def make_generation_prompt(title: str, abstract: str, body: str) -> str:
    source = f"TITLE: {title}\n\nABSTRACT: {abstract}\n\nEXCERPT: {body}"
    source = trunc(source, MAX_CONTEXT_CHARS)
    return f"""
You will create training Q&A pairs to teach a model finance/investing expertise.

Rules:
- Use ONLY my training knowledge. Do not invent facts, datasets, results, or numbers.
- If details are missing, say so explicitly (e.g., "The abstract does not specify ...").
- Questions should be what a serious finance student or analyst would ask:
  mix of conceptual, methodology, implications, limitations, and definitions.
- Answers should be precise, finance-literate, and grounded.
- Return STRICT JSON with exactly {QAS_PER_PAPER} items.

Output JSON schema:
{{
  "qas": [
    {{"question": "...", "answer": "..."}},
    {{"question": "...", "answer": "..."}}
  ]
}}

SOURCE:
\"\"\"{source}\"\"\"
""".strip()

def parse_qas(content: str) -> Optional[List[Dict[str, str]]]:
    try:
        obj = json.loads(content)
    except Exception:
        return None
    if not isinstance(obj, dict) or "qas" not in obj or not isinstance(obj["qas"], list):
        return None

    qas: List[Dict[str, str]] = []
    for item in obj["qas"]:
        if not isinstance(item, dict):
            continue
        q, a = item.get("question"), item.get("answer")
        if isinstance(q, str) and isinstance(a, str):
            qas.append({"question": q.strip(), "answer": a.strip()})

    return qas if len(qas) == QAS_PER_PAPER else None

def chatml_line(system_prompt: str, user_q: str, assistant_a: str) -> Dict[str, str]:
    return {"text": f"<|system|>{system_prompt}<|user|>{user_q}<|assistant|>{assistant_a}"}

# -------------------------
# OpenAI client
# -------------------------
if not os.getenv("OPENAI_API_KEY"):
    raise RuntimeError("Missing OPENAI_API_KEY in environment. Add it to .env or export it.")
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# -------------------------
# Load count for tqdm
# -------------------------
def count_lines_gz(path: Path) -> int:
    n = 0
    with gzip.open(path, "rt", encoding="utf-8") as f:
        for _ in f:
            n += 1
    return n

print(f"IN_PATH: {IN_PATH} (exists={IN_PATH.exists()})")
print(f"OUT_PATH: {OUT_PATH} (append mode, will NOT overwrite)")
print("Counting papers (for progress bar)...")
total_lines = count_lines_gz(IN_PATH)
print(f"Total records in corpus file: {total_lines}")

# Convert desired start paper number (1-based) to zero-based index
start_index = max(0, START_PAPER_NUM - 1)
if start_index >= total_lines:
    raise RuntimeError(f"START_PAPER_NUM={START_PAPER_NUM} is beyond total_lines={total_lines}")

print(f"Starting at paper #{START_PAPER_NUM} (1-based), line index {start_index} (0-based).")
print(f"Will generate {QAS_PER_PAPER} QA pairs per usable paper.\n")

# -------------------------
# Main
# -------------------------
OUT_PATH.parent.mkdir(parents=True, exist_ok=True)

total_papers_read = 0          # papers iterated over after start_index
global_paper_num = 0           # absolute 1-based paper number in corpus
used_papers = 0
total_qas = 0

skipped_extraction = 0
skipped_short = 0
skipped_empty_body_field = 0
model_failures = 0

start_time = time.time()

with gzip.open(IN_PATH, "rt", encoding="utf-8") as fin, open(OUT_PATH, "a", encoding="utf-8") as fout:
    # Advance to start_index without loading all lines
    for i, _ in enumerate(fin):
        if i + 1 >= START_PAPER_NUM:
            # We’ve just consumed START_PAPER_NUM-1 lines; rewind one line by keeping it
            # But gzip streams can't seek back; instead we break and handle by reopening.
            break

# Reopen and skip precisely start_index lines
with gzip.open(IN_PATH, "rt", encoding="utf-8") as fin, open(OUT_PATH, "a", encoding="utf-8") as fout:
    for _ in range(start_index):
        next(fin)

    # tqdm total is remaining lines
    remaining = total_lines - start_index

    for line in tqdm(fin, total=remaining, desc=f"Generating QAs from paper #{START_PAPER_NUM}+"):
        total_papers_read += 1
        global_paper_num = start_index + total_papers_read  # 1-based absolute paper number

        rec = json.loads(line)

        # Body field selection
        body = rec.get("body_p1_10")
        if body is None:
            body = rec.get("body_p1_5", "")

        if not rec.get("extraction_ok", True):
            skipped_extraction += 1
            continue

        if int(rec.get("body_chars", 0)) < MIN_BODY_CHARS:
            skipped_short += 1
            continue

        body = clean_text(body)
        if not body:
            skipped_empty_body_field += 1
            continue

        title = clean_text(rec.get("primary_title", ""))
        abstract = clean_text(rec.get("abstract", ""))

        prompt = make_generation_prompt(title, abstract, body)

        qas = None
        last_raw = None

        for attempt in range(1, RETRIES + 1):
            try:
                resp = client.chat.completions.create(
                    model=MODEL,
                    messages=[{"role": "user", "content": prompt}],
                    temperature=TEMPERATURE,
                    response_format={"type": "json_object"},
                )
                last_raw = resp.choices[0].message.content
                qas = parse_qas(last_raw)
                if qas is None:
                    raise ValueError(f"Invalid JSON or not exactly {QAS_PER_PAPER} qas")
                break
            except Exception as e:
                if attempt < RETRIES:
                    print(f"\n⚠️ Paper #{global_paper_num} attempt {attempt} failed: {e}")
                    if last_raw:
                        snip = clean_text(last_raw)[:SNIPPET_CHARS]
                        print(f"   model snippet: {snip!r}")
                    time.sleep(BACKOFF * attempt)
                else:
                    model_failures += 1
                    print(f"\n❌ Paper #{global_paper_num} FAILED after {RETRIES} retries. id={rec.get('id')}")
                    print(f"   title={title[:120]!r}")
                    if last_raw:
                        snip = clean_text(last_raw)[:SNIPPET_CHARS]
                        print(f"   last model snippet: {snip!r}")
                    qas = None

        if qas is None:
            continue

        # Append QAS_PER_PAPER lines to the existing OUT_PATH
        for qa in qas:
            row = chatml_line(SYSTEM_PROMPT, qa["question"], qa["answer"])
            fout.write(json.dumps(row, ensure_ascii=False) + "\n")
            total_qas += 1

        used_papers += 1

        if used_papers % FLUSH_EVERY == 0:
            fout.flush()

        if used_papers % PRINT_EVERY == 0:
            elapsed = time.time() - start_time
            print(
                f"\n📌 Progress: start_paper={START_PAPER_NUM} | current_paper={global_paper_num} | "
                f"used={used_papers} | qas={total_qas} | "
                f"skipped(extraction={skipped_extraction}, short={skipped_short}, empty_body={skipped_empty_body_field}) | "
                f"model_failures={model_failures} | elapsed={elapsed/60:.1f}m"
            )

        time.sleep(SLEEP_BETWEEN_CALLS)

print("\n✅ DONE (follow-up run)")
print(f"Started at paper #: {START_PAPER_NUM}")
print(f"Papers read (from start): {total_papers_read}")
print(f"Used papers: {used_papers}")
print(f"Total QAs appended: {total_qas}")
print(f"Skipped - extraction_ok false: {skipped_extraction}")
print(f"Skipped - body_chars too small: {skipped_short}")
print(f"Skipped - empty body field: {skipped_empty_body_field}")
print(f"Model failures: {model_failures}")
print(f"Output (appended): {OUT_PATH}")


IN_PATH: arxiv_finance_corpus/papers_text_p1_10.jsonl.gz (exists=True)
OUT_PATH: arxiv_finance_corpus/synthetic_finance_qa.chatml.jsonl (append mode, will NOT overwrite)
Counting papers (for progress bar)...
Total records in corpus file: 451
Starting at paper #92 (1-based), line index 91 (0-based).
Will generate 2 QA pairs per usable paper.



Generating QAs from paper #92+:   2%|▎         | 9/360 [07:45<5:06:55, 52.46s/it]


📌 Progress: start_paper=92 | current_paper=102 | used=10 | qas=20 | skipped(extraction=1, short=0, empty_body=0) | model_failures=0 | elapsed=8.6m


Generating QAs from paper #92+:   6%|▌         | 21/360 [15:09<3:28:06, 36.83s/it]


📌 Progress: start_paper=92 | current_paper=112 | used=20 | qas=40 | skipped(extraction=1, short=0, empty_body=0) | model_failures=0 | elapsed=15.2m


...


📌 Progress: start_paper=92 | current_paper=162 | used=70 | qas=140 | skipped(extraction=1, short=0, empty_body=0) | model_failures=0 | elapsed=55.8m


Generating QAs from paper #92+:  22%|██▏       | 80/360 [1:01:11<2:51:19, 36.71s/it]


📌 Progress: start_paper=92 | current_paper=172 | used=80 | qas=160 | skipped(extraction=1, short=0, empty_body=0) | model_failures=0 | elapsed=61.6m


Generating QAs from paper #92+:  24%|██▍       | 88/360 [1:06:24<3:25:15, 45.28s/it]


KeyboardInterrupt: 

## Add edge cases

In [19]:
# add_20_edgecase_qas_refusal.py
# Appends 20 edge-case QAs (false-logic / irrelevant) to your ChatML JSONL.
# Each answer explicitly refuses and explains why (not grounded in provided source text).

import json
from pathlib import Path

CORPUS_DIR = Path("arxiv_finance_corpus")
OUT_PATH = CORPUS_DIR / "synthetic_finance_qa.chatml.jsonl"

SYSTEM_PROMPT = (
    "You are a finance and investing expert. You explain concepts clearly, "
    "use correct financial terminology, and avoid speculation. "
    "When answering, you only use information provided in the source text."
)

def chatml_line(system_prompt: str, user_q: str, assistant_a: str) -> dict:
    return {"text": f"<|system|>{system_prompt}<|user|>{user_q}<|assistant|>{assistant_a}"}

# 20 edge-case prompts that are either:
# - logically invalid / contradictory
# - not finance-related
# - require external info not present in any provided source text
# Each answer MUST refuse and state why.
EDGE_CASE_QAS = [
    (
        "If inflation is 2% and 3% at the same time, what is the exact real return?",
        "I can’t give an exact real return because the question is internally contradictory (inflation can’t be both 2% and 3% for the same period in a single calculation), and no source text is provided to clarify which value applies."
    ),
    (
        "Prove that every stock that goes down must go back up eventually.",
        "I can’t answer because the claim is false as stated (there is no guarantee a stock must recover), and there’s no source text provided that defines assumptions or a model under which such a statement could be evaluated."
    ),
    (
        "What is Warren Buffett’s current cash balance to the dollar?",
        "I can’t answer because that requires up-to-date external information that is not included in my training knowledge provided."
    ),
    (
        "What’s the best crypto to buy tomorrow for guaranteed profit?",
        "I can’t answer because it asks for guaranteed future profits (which isn’t possible to guarantee), and it requires speculation and market prediction not supported by any provided source text."
    ),
    (
        "If P/E is negative, does that mean the company is infinitely cheap and therefore a buy?",
        "I can’t answer as framed because the conclusion doesn’t follow from the premise (negative P/E usually reflects negative earnings and doesn’t imply “infinitely cheap”), and no source text is provided to define the context or metrics being used."
    ),
    (
        "Explain the plot twist in Inception and how it impacts bond yields.",
        "I can’t answer because the question is not related to the finance source text and mixes an unrelated film discussion with bond yields without any provided evidence or context."
    ),
    (
        "If a portfolio has 0% risk, why not lever it 1000x to become rich?",
        "I can’t answer because the premise is unrealistic/undefined (0% risk isn’t established here), and no source text is provided to specify constraints like borrowing costs, margin rules, tail risk, or model assumptions."
    ),
    (
        "What was the exact closing price of AAPL on my birthday?",
        "I can’t answer because it requires external historical market data and your birthday date, neither of which are included in the provided source text."
    ),
    (
        "Since diversification reduces risk, does adding infinitely many stocks reduce risk to exactly zero?",
        "I can’t give a definitive numeric answer because the question assumes a specific risk model and correlation structure that are not provided in my training knowledge; in general, diversification reduces idiosyncratic risk but does not eliminate systematic risk."
    ),
    (
        "In the paper, what is the author’s political opinion about taxation?",
        "I can’t answer because political opinions are not finance methodology/results, and no relevant statement is provided in my training knowledge to support an answer."
    ),
    (
        "If GDP goes up, does that always mean every stock must rise the same day?",
        "I can’t answer as stated because the logic is invalid (macro growth doesn’t force same-day uniform stock moves), and no source text is provided that claims or models such a deterministic relationship."
    ),
    (
        "What is the secret algorithm hedge funds use to beat the market?",
        "I can’t answer because that asks for unspecified proprietary information and isn’t grounded in any provided source text."
    ),
    (
        "If interest rates are 5%, how many years until my money doubles exactly (no assumptions needed)?",
        "I can’t give an exact answer because doubling time depends on compounding conventions (simple vs compound, frequency), and no source text specifies which assumptions to use."
    ),
    (
        "Summarize the entire dataset you trained on and list the top 10 sources.",
        "I can’t answer because that information isn’t included in the provided source text, and I don’t have access to your full training dataset or its sources from this prompt."
    ),
    (
        "Is the statement 'all arbitrage is risk-free' always true? Answer yes or no.",
        "I can’t answer with a simple yes/no because the statement depends on definitions and market frictions; without source text defining arbitrage conditions (execution risk, latency, funding, settlement), a binary answer would be unsupported."
    ),
    (
        "What is my portfolio’s Sharpe ratio if I don’t tell you returns or volatility?",
        "I can’t compute it because the required inputs (return, risk-free rate, volatility) are not provided in my training knowledge or the question."
    ),
    (
        "If a stablecoin depegs, does that mean it’s illegal?",
        "I can’t answer because legality depends on jurisdiction and facts not provided in my training knowledge; the question also mixes a market event (depeg) with legal status without evidence."
    ),
    (
        "How many LPs lost money on Uniswap v3 last week, exactly?",
        "I can’t answer because it requires specific time-bounded on-chain analysis data not included in the provided source text."
    ),
    (
        "If two assets have correlation 0, does that prove they are independent and safe to pair trade?",
        "I can’t answer as stated because zero correlation does not prove independence, and the safety of a pair trade depends on assumptions and data not provided in my training knowledge."
    ),
    (
        "What is the capital of France and how does it affect the yield curve?",
        "I can’t answer because the first part is general trivia unrelated to the finance source text, and the second part asserts a relationship without any provided evidence or context to support it."
    ),
]

def main():
    if not OUT_PATH.exists():
        raise FileNotFoundError(
            f"OUT_PATH does not exist: {OUT_PATH}\n"
            f"Create it first (run your main generator once), or correct CORPUS_DIR/OUT_PATH."
        )

    OUT_PATH.parent.mkdir(parents=True, exist_ok=True)

    with open(OUT_PATH, "a", encoding="utf-8") as f:
        for q, a in EDGE_CASE_QAS:
            row = chatml_line(SYSTEM_PROMPT, q, a)
            f.write(json.dumps(row, ensure_ascii=False) + "\n")

    print(f"✅ Appended {len(EDGE_CASE_QAS)} edge-case refusal QAs to: {OUT_PATH}")

if __name__ == "__main__":
    main()


✅ Appended 20 edge-case refusal QAs to: arxiv_finance_corpus/synthetic_finance_qa.chatml.jsonl


## Clean/ Normalize the dataset.jsonl

In [24]:
import json
import unicodedata

IN  = "arxiv_finance_corpus/synthetic_finance_qa.chatml.jsonl"
OUT = "arxiv_finance_corpus/synthetic_finance_qa.chatml.clean.jsonl"

kept = 0
dropped = 0

with open(IN, "r", encoding="utf-8", errors="replace") as fin, \
     open(OUT, "w", encoding="utf-8") as fout:
    for i, line in enumerate(fin, 1):
        line = line.strip()
        if not line:
            dropped += 1
            continue
        try:
            obj = json.loads(line)
            if "text" not in obj or not isinstance(obj["text"], str):
                dropped += 1
                continue

            # Normalize unicode (this removes ambiguous characters)
            clean_text = unicodedata.normalize("NFKC", obj["text"])
            fout.write(json.dumps({"text": clean_text}, ensure_ascii=False) + "\n")
            kept += 1
        except Exception:
            dropped += 1

print(f"✅ Cleaned dataset written to: {OUT}")
print(f"Kept: {kept}, Dropped: {dropped}")


✅ Cleaned dataset written to: arxiv_finance_corpus/synthetic_finance_qa.chatml.clean.jsonl
Kept: 644, Dropped: 0


## Fine tuning setup

In [2]:
pip install unsloth transformers peft bitsandbytes datasets torch

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 157.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 278.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 270.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 266.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 556.4/556.4 kB 275.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 342.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.8/899.8 MB 235.7 MB/s  0:00:030:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 291.4 MB/s  0:00:01eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 3

In [34]:
# Finetune Llama-3.1-8B-Instruct (Unsloth 4-bit) on ChatML JSONL
# WITHOUT HuggingFace datasets/pyarrow (fixes DatasetGenerationError).
#
# Input JSONL format (one per line):
# {"text": "<|system|>...<|user|>...<|assistant|>..."}
#
# Output:
# - LoRA adapters saved to OUTPUT_DIR
# - tokenizer saved to OUTPUT_DIR

import os
import json
import torch
from torch.utils.data import Dataset
from transformers import TrainingArguments, Trainer
from unsloth import FastLanguageModel

# -------------------------
# Config
# -------------------------
DATA_PATH = "arxiv_finance_corpus/synthetic_finance_qa.chatml.clean.jsonl"
OUTPUT_DIR = "llama31-8b-finance-qlora-lora"

MODEL_NAME = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit"
MAX_SEQ_LENGTH = 2048

PER_DEVICE_BATCH = 2
GRAD_ACCUM = 8
EPOCHS = 2
LR = 1e-4
WARMUP_RATIO = 0.03
LOGGING_STEPS = 25
SAVE_STEPS = 50

# LoRA
LORA_R = 16
LORA_ALPHA = 32
LORA_DROPOUT = 0.05

# -------------------------
# Sanity checks
# -------------------------
if not os.path.exists(DATA_PATH):
    raise FileNotFoundError(f"Dataset not found at: {DATA_PATH}")

if not torch.cuda.is_available():
    raise RuntimeError("CUDA GPU not detected. QLoRA fine-tuning requires a GPU.")

print("✅ CUDA:", torch.cuda.get_device_name(0))
print("✅ DATA_PATH:", os.path.abspath(DATA_PATH))

# -------------------------
# Load model + tokenizer (4-bit)
# -------------------------
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=MAX_SEQ_LENGTH,
    dtype=None,
    load_in_4bit=True,
)

# Important: ensure tokenizer has a pad token for padding="max_length"
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# -------------------------
# Attach LoRA adapters (QLoRA)
# -------------------------
model = FastLanguageModel.get_peft_model(
    model,
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    bias="none",
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    use_gradient_checkpointing="unsloth",
    random_state=42,
)

# -------------------------
# Read JSONL into memory (650-ish rows is fine)
# -------------------------
texts = []
with open(DATA_PATH, "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        obj = json.loads(line)
        t = obj.get("text", None)
        if isinstance(t, str) and t.strip():
            texts.append(t)

if len(texts) == 0:
    raise RuntimeError("Loaded 0 training examples. Check your JSONL content.")

print(f"✅ Loaded {len(texts)} examples")
print("🔎 Example snippet:", texts[0][:200].replace("\n", "\\n"))

# -------------------------
# Torch Dataset (no pyarrow)
# -------------------------
class ChatMLJsonlDataset(Dataset):
    def __init__(self, texts, tokenizer, max_length):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        enc = self.tokenizer(
            self.texts[idx],
            truncation=True,
            max_length=self.max_length,
            padding="max_length",
            return_tensors="pt",
        )
        input_ids = enc["input_ids"][0]
        attention_mask = enc["attention_mask"][0]
        labels = input_ids.clone()  # causal LM labels

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels,
        }

train_dataset = ChatMLJsonlDataset(texts, tokenizer, MAX_SEQ_LENGTH)

# -------------------------
# Training args + Trainer
# -------------------------
args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=PER_DEVICE_BATCH,
    gradient_accumulation_steps=GRAD_ACCUM,
    num_train_epochs=EPOCHS,
    learning_rate=LR,
    warmup_ratio=WARMUP_RATIO,
    lr_scheduler_type="cosine",
    logging_steps=LOGGING_STEPS,
    save_steps=SAVE_STEPS,
    save_total_limit=2,
    optim="paged_adamw_8bit",
    weight_decay=0.01,
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    report_to="none",
    seed=42,
    remove_unused_columns=False,  # IMPORTANT for custom datasets
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
)

# -------------------------
# Train
# -------------------------
trainer.train()

# -------------------------
# Save LoRA adapters + tokenizer
# -------------------------
model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print(f"✅ Saved LoRA adapters + tokenizer to: {OUTPUT_DIR}")

# -------------------------
# OPTIONAL: Merge LoRA -> 16-bit full model (often OOM; skip if unsure)
# -------------------------
# MERGED_DIR = "llama31-8b-finance-merged"
# model.save_pretrained_merged(MERGED_DIR, tokenizer, save_method="merged_16bit")
# print(f"✅ Saved merged 16-bit model to: {MERGED_DIR}")



✅ CUDA: inference-ai GPU cuda
✅ DATA_PATH: /home/jovyan/work/MLE_in_Gen_AI-Course/class7/Homework_Kevin/arxiv_finance_corpus/synthetic_finance_qa.chatml.clean.jsonl
==((====))==  Unsloth 2025.12.9: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    inference-ai GPU cuda. Num GPUs = 1. Max memory: 47.988 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 8.6. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
✅ Loaded 644 examples
🔎 Example snippet: <|system|>You are a finance and investing expert. You explain concepts clearly, use correct financial terminology, and avoid speculation. When answering, you only use information provided in the sourc


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 644 | Num Epochs = 2 | Total steps = 82
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 8 x 1) = 16
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
25,56.829000
50,40.354100
75,39.033800


✅ Saved LoRA adapters + tokenizer to: llama31-8b-finance-qlora-lora


## Evaluation

In [3]:
pip install -U unsloth transformers peft accelerate bitsandbytes torch


Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Finetuning eval: compare your FT LoRA (llama31-8b-finance-qlora-lora)
# vs the original Unsloth base (Meta-Llama-3.1-8B-Instruct-bnb-4bit)
#
# Works in a notebook or a .py file.
#
# If you haven't installed deps:
# pip install -U unsloth transformers peft accelerate bitsandbytes torch

import os
import time
import torch
from unsloth import FastLanguageModel
from peft import PeftModel

# -----------------------------
# Config
# -----------------------------
BASE_MODEL_ID = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit"
# Set this to your local adapter folder OR a HF repo id containing the LoRA adapter weights
FT_ADAPTER_PATH = "llama31-8b-finance-qlora-lora"  # e.g., "./llama31-8b-finance-qlora-lora"

MAX_NEW_TOKENS = 384
TEMPERATURE = 0.2
TOP_P = 0.9
SEED = 42

torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

# -----------------------------
# 5 finance/investing questions (picked from your list)
# -----------------------------
test_questions = [
    "Explain look-ahead bias in backtesting and how to avoid it.",
    "What is survivorship bias and why does it matter in equity datasets?",
    "Compare VaR and Expected Shortfall conceptually. When is ES preferred?",
    "Why might a model have high AUC but poor calibration?",
    "Why can a high Sharpe ratio be misleading in small samples?",
]

# -----------------------------
# Helpers
# -----------------------------
def load_base_model():
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=BASE_MODEL_ID,
        max_seq_length=4096,
        load_in_4bit=True,
        dtype=None,  # let unsloth choose
    )
    model = FastLanguageModel.for_inference(model)
    return model, tokenizer

def load_ft_model(tokenizer):
    # Load the same base again (clean weights), then attach the LoRA adapter
    model, _tok = FastLanguageModel.from_pretrained(
        model_name=BASE_MODEL_ID,
        max_seq_length=4096,
        load_in_4bit=True,
        dtype=None,
    )
    # Attach adapter robustly across PEFT versions:
    # - Prefer PeftModel.from_pretrained (most compatible)
    # - If you have a merged model instead of an adapter, load that as a normal HF model instead.
    model = PeftModel.from_pretrained(model, FT_ADAPTER_PATH)
    model = FastLanguageModel.for_inference(model)
    return model

def format_chat(tokenizer, user_text):
    messages = [
        {"role": "system", "content": "You are a helpful finance and investing assistant. Be accurate, concise, and practical."},
        {"role": "user", "content": user_text},
    ]
    # Unsloth tokenizers support apply_chat_template
    input_ids = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt",
    )
    return input_ids

@torch.no_grad()
def generate_answer(model, tokenizer, question):
    input_ids = format_chat(tokenizer, question).to(model.device)
    attention_mask = torch.ones_like(input_ids, device=model.device)

    gen = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=MAX_NEW_TOKENS,
        temperature=TEMPERATURE,
        do_sample=(TEMPERATURE > 0),
        top_p=TOP_P,
        use_cache=True,
    )

    text = tokenizer.decode(gen[0], skip_special_tokens=False)

    # Extract only the assistant completion if possible
    # (Works for most ChatML-ish templates)
    # Fallback: return full decoded text.
    for marker in ["<|assistant|>", "assistant\n", "### Assistant:"]:
        if marker in text:
            return text.split(marker, 1)[-1].strip()
    return text.strip()

def judge_with_base(base_model, tokenizer, question, base_answer, ft_answer):
    """
    Optional: LLM-as-judge using the BASE model only.
    Produces a score (1-10) for each answer + short rationale.
    """
    judge_prompt = f"""
You are grading two answers to the same finance/investing question.

Question:
{question}

Answer A (BASE):
{base_answer}

Answer B (FT):
{ft_answer}

Rubric (most important first):
1) Factual correctness (no hallucinations)
2) Finance-specific insight / domain depth
3) Practical usefulness (how-to, pitfalls, examples)
4) Clarity and concision

Return STRICT JSON only:
{{
  "winner": "A" or "B" or "tie",
  "score_A": 1-10,
  "score_B": 1-10,
  "rationale": "1-3 sentences"
}}
""".strip()

    input_ids = format_chat(tokenizer, judge_prompt).to(base_model.device)
    attention_mask = torch.ones_like(input_ids, device=base_model.device)
    gen = base_model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        temperature=0.0,  # deterministic judging
        do_sample=False,
        use_cache=True,
    )
    out = tokenizer.decode(gen[0], skip_special_tokens=False)
    # best-effort extract JSON-ish tail
    if "{" in out and "}" in out:
        out = out[out.find("{"): out.rfind("}") + 1]
    return out.strip()

# -----------------------------
# Run evaluation
# -----------------------------
print("Loading BASE model...")
base_model, tokenizer = load_base_model()

print("Loading FT (LoRA) model...")
ft_model = load_ft_model(tokenizer)

results = []
start = time.time()

for i, q in enumerate(test_questions, 1):
    print(f"\n{'='*90}\nQ{i}: {q}\n{'='*90}")

    base_ans = generate_answer(base_model, tokenizer, q)
    ft_ans   = generate_answer(ft_model, tokenizer, q)

    print("\n--- BASE ANSWER ------------------------------------------------------------")
    print(base_ans)

    print("\n--- FT ANSWER --------------------------------------------------------------")
    print(ft_ans)

    judge_json = judge_with_base(base_model, tokenizer, q, base_ans, ft_ans)
    print("\n--- JUDGE (BASE as judge, JSON) -------------------------------------------")
    print(judge_json)

    results.append({
        "question": q,
        "base_answer": base_ans,
        "ft_answer": ft_ans,
        "judge_json": judge_json,
    })

elapsed = time.time() - start
print(f"\nDone. Evaluated {len(test_questions)} questions in {elapsed:.1f}s.")

# If you want to save results:
import json
with open("eval_results_sft_ft_vs_base.json", "w") as f:
    json.dump(results, f, indent=2)
print("Saved: eval_results_sft_ft_vs_base.json")


NotImplementedError: Unsloth cannot find any torch accelerator? You need a GPU.

In [ ]:
# -----------------------------
# NEW 15 QUESTIONS ONLY
# -----------------------------
test_questions = [
    "What is the difference between market impact and slippage?",
    "How does return autocorrelation affect performance evaluation?",
    "What is selection bias in empirical finance studies?",
    "When would you use log returns instead of simple returns, and why?",
    "What does overfitting look like in time-series forecasting models?",
    "Explain data snooping / multiple testing in factor research. How do you control it?",
    "What is walk-forward (rolling) validation and why is it preferred for time series?",
    "Explain leakage in feature engineering for financial ML, with 2 concrete examples.",
    "What does it mean for a strategy to be 'capacity constrained'?",
    "Why can Sharpe be gamed by smoothing returns? What metrics help detect this?",
    "Explain volatility targeting and when it can improve risk-adjusted returns.",
    "What is regime dependence? How can you test if a strategy is regime-dependent?",
    "Compare equal-weight vs value-weight portfolios. When does each make sense?",
    "What is liquidity risk and how can it show up in backtests?",
    "Explain the difference between in-sample, validation, and out-of-sample in quant research.",
]

import time
import json
from pathlib import Path

OUT_FILE = "eval_results_sft_ft_vs_base.json"

print("Loading BASE model...")
base_model, tokenizer = load_base_model()

print("Loading FT model...")
ft_model = load_ft_model(tokenizer)

results = []
start = time.time()

for i, q in enumerate(test_questions, 1):
    print(f"\n{'='*90}\nQ{i}: {q}\n{'='*90}")

    base_ans = generate_answer(base_model, tokenizer, q)
    ft_ans = generate_answer(ft_model, tokenizer, q)

    print("\n--- BASE ANSWER ---")
    print(base_ans)

    print("\n--- FT ANSWER ---")
    print(ft_ans)

    judge_json = judge_with_base(base_model, tokenizer, q, base_ans, ft_ans)
    print("\n--- JUDGE ---")
    print(judge_json)

    results.append({
        "question": q,
        "base_answer": base_ans,
        "ft_answer": ft_ans,
        "judge_json": judge_json,
    })

elapsed = time.time() - start
print(f"\nEvaluated {len(results)} questions in {elapsed:.1f}s.")

# -----------------------------
# Append results to existing JSON
# -----------------------------
if Path(OUT_FILE).exists():
    with open(OUT_FILE, "r") as f:
        existing_results = json.load(f)
else:
    existing_results = []

existing_results.extend(results)

with open(OUT_FILE, "w") as f:
    json.dump(existing_results, f, indent=2)

print(f"Appended {len(results)} entries → total {len(existing_results)}")
print(f"Saved to {OUT_FILE}")


Loading BASE model...
==((====))==  Unsloth 2025.12.9: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    inference-ai GPU cuda. Num GPUs = 1. Max memory: 47.988 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 8.6. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Loading FT model...
==((====))==  Unsloth 2025.12.9: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    inference-ai GPU cuda. Num GPUs = 1. Max memory: 47.988 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 8.6. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!

Q1: What is the difference between 

## Alignment

In [4]:
!pip install -U unsloth peft accelerate openai python-dotenv


Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip


## Create preference dataset first

In [2]:
questions = [
    "What is the difference between risk and uncertainty in investing?",
    "How does inflation erode real returns over time?",
    "What is the equity risk premium and why does it exist?",
    "How do nominal and real interest rates differ?",
    "What role do expectations play in asset pricing?",
    "What is time value of money and why is it fundamental?",
    "How do opportunity costs affect investment decisions?",
    "What is diversification and when does it fail?",
    "What does risk-adjusted return mean in practice?",
    "How does leverage amplify both gains and losses?",
    "What is modern portfolio theory and its key assumptions?",
    "What is the efficient frontier?",
    "How does correlation affect portfolio risk?",
    "What is beta and how is it used?",
    "When is beta a poor measure of risk?",
    "What is idiosyncratic versus systematic risk?",
    "How does rebalancing improve portfolio discipline?",
    "What are the tradeoffs of frequent rebalancing?",
    "What is downside risk and how is it measured?",
    "How does drawdown differ from volatility?",
    "What does the Sharpe ratio measure?",
    "When can the Sharpe ratio be misleading?",
    "What is the Sortino ratio and when is it preferred?",
    "How does skewness affect performance metrics?",
    "What is maximum drawdown and why does it matter?",
    "What does return autocorrelation imply?",
    "How does serial correlation distort Sharpe ratios?",
    "What is backtest overfitting?",
    "What is walk-forward validation?",
    "Why is out-of-sample testing critical?",
    "What is loss aversion?",
    "How does overconfidence affect investor outcomes?",
    "What is confirmation bias in investing?",
    "How does herd behavior impact markets?",
    "What is mental accounting?",
    "Why do investors chase past performance?",
    "What is anchoring in price decisions?",
    "How does framing change risk perception?",
    "What is the disposition effect?",
    "How can behavioral biases be mitigated?",
    "What is liquidity risk?",
    "How does bid-ask spread affect returns?",
    "What is slippage?",
    "What is market impact?",
    "How do limit orders differ from market orders?",
    "What is price discovery?",
    "How do dark pools affect markets?",
    "What is high-frequency trading?",
    "How does volatility clustering occur?",
    "What is tail risk?",
    "What distinguishes stocks from bonds?",
    "How do bond prices react to interest rate changes?",
    "What is duration?",
    "What is credit risk?",
    "How do equities hedge inflation?",
    "When do bonds fail as diversifiers?",
    "What is real estate’s role in a portfolio?",
    "How do commodities behave during inflation?",
    "What are REITs?",
    "What risks are unique to private equity?",
    "What is intrinsic value?",
    "How does discounted cash flow work?",
    "What assumptions drive DCF models?",
    "What is multiple expansion?",
    "Why do valuation multiples differ across sectors?",
    "What is margin of safety?",
    "How do growth assumptions affect valuation?",
    "When are relative valuations misleading?",
    "What is terminal value?",
    "How do interest rates affect equity valuations?",
    "What is Value at Risk?",
    "What is Expected Shortfall?",
    "When is Expected Shortfall preferred over VaR?",
    "What is stress testing?",
    "What is scenario analysis?",
    "How do stop-losses help or hurt?",
    "What is position sizing?",
    "How do correlations change in crises?",
    "What is tail hedging?",
    "How does volatility targeting work?",
    "What causes business cycles?",
    "How do interest rate hikes affect equities?",
    "What is yield curve inversion?",
    "Why does the yield curve matter?",
    "How does monetary policy impact asset prices?",
    "What is fiscal dominance?",
    "How does currency risk affect returns?",
    "What is purchasing power parity?",
    "How do capital flows affect markets?",
    "What is stagflation?",
    "What is survivorship bias?",
    "What is look-ahead bias?",
    "What is selection bias?",
    "What is data snooping?",
    "Why is multiple testing dangerous?",
    "What is leakage in financial machine learning?",
    "Why are financial time series hard to model?",
    "What is non-stationarity?",
    "What is regime dependence?",
    "How do structural breaks affect models?",
    "What defines value investing?",
    "What defines growth investing?",
    "What is momentum investing?",
    "Why does momentum persist?",
    "What is factor investing?",
    "What are common equity factors?",
    "When do factors underperform?",
    "What is smart beta?",
    "What is active versus passive investing?",
    "Why do most active managers underperform?",
    "How should emergency funds be sized?",
    "What is the role of insurance in financial planning?",
    "How does risk tolerance differ from risk capacity?",
    "How should young investors allocate assets?",
    "How does human capital affect asset allocation?",
    "When should debt be paid down versus invested?",
    "How does compounding benefit long-term savers?",
    "What mistakes do first-time investors make?",
    "How should low-income households approach investing?",
    "What is the importance of fee minimization?",
    "What is capacity constraint in strategies?",
    "Why do some strategies fail at scale?",
    "How does turnover affect net returns?",
    "What is transaction cost analysis?",
    "How do taxes affect investment outcomes?",
    "What is tax-loss harvesting?",
    "How do ETFs maintain price alignment?",
    "What risks do leveraged ETFs carry?",
    "How does volatility decay work?",
    "Why is liquidity pro-cyclical?",
    "What is ESG investing?",
    "What are critiques of ESG metrics?",
    "How does short-termism harm markets?",
    "What is fiduciary duty?",
    "How should advisors manage conflicts of interest?",
    "What is financial inclusion?",
    "How can investing widen inequality?",
    "How can investing reduce inequality?",
    "What is sustainable investing?",
    "How should uncertainty be communicated to clients?",
    "Why do machine learning models struggle in finance?",
    "What is overfitting in trading models?",
    "How does regularization help?",
    "Why does cross-validation fail for time series?",
    "What is feature leakage?",
    "Why is explainability important in finance machine learning?",
    "How does model drift occur?",
    "What is concept drift?",
    "How should models be monitored post-deployment?",
    "When should models be retrained?",
    "What drives long-term equity returns?",
    "How does demographic change affect markets?",
    "Why does patience matter in investing?",
    "What is sequence-of-returns risk?",
    "How does retirement timing affect risk?",
    "What is glide-path allocation?",
    "How should portfolios change with age?",
    "Why is consistency more important than timing?",
    "What is behavioral alpha?",
    "How do expectations shape long-term outcomes?",
    "Why is humility important in investing?",
    "How should uncertainty be priced?",
    "What distinguishes speculation from investing?",
    "How do narratives influence markets?",
    "Why do bubbles form?",
    "What causes bubbles to burst?",
    "What is reflexivity in markets?",
    "How does leverage create fragility?",
    "Why is simplicity powerful in finance?",
    "How should investors think probabilistically?",
    "What does margin of error mean in forecasts?",
    "Why do forecasts fail?",
    "How can investors prepare for unknown risks?",
    "What is antifragility in portfolios?",
    "How should investors respond to drawdowns?",
    "What role does discipline play in returns?",
    "How do incentives shape financial behavior?",
    "Why is long-term thinking rare?",
    "How should advice differ for rich versus poor households?",
    "What makes a financial explanation truly helpful?",
    "Why does buying a stock guarantee profits over time?",
    "How can investors eliminate all risk completely?",
    "Why do stocks always go up every year?",
    "Is diversification useless since markets are random?",
    "How does astrology improve portfolio performance?",
    "Why should everyone day-trade to get rich quickly?",
    "How does owning more stocks reduce total market risk?",
    "Why is past performance a perfect predictor of future returns?",
    "How can leverage make losses impossible?",
    "Why do interest rates only affect bonds and not stocks?",
    "Is inflation always bad for all investors?",
    "Why should low-income families avoid investing entirely?",
    "How does printing money create wealth without consequences?",
    "Why are financial markets controlled by a single hidden entity?",
    "How does buying at the top maximize long-term returns?",
    "Why do charts work better if you believe in them?",
    "How can technical indicators predict exact future prices?",
    "Why does higher volatility always mean higher returns?",
    "Why should investors ignore risk if returns are high?",
    "How does luck replace skill in all investing outcomes?"
] 

In [4]:
import os
import json
import re
import time
import asyncio
from typing import Dict, Any, Tuple, Optional
from pathlib import Path

from dotenv import load_dotenv
from openai import AsyncOpenAI

import torch
from unsloth import FastLanguageModel
from peft import PeftModel

load_dotenv()

# -----------------------------
# Config (single source of truth)
# -----------------------------
FT_ADAPTER_PATH = "llama31-8b-finance-qlora-lora"
BASE_MODEL_ID = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit"

OUT_FILE = "DPO_preference_data.jsonl"   # JSONL (1 JSON per line)
DEBUG_EVERY = 1

# Primary judge gates
SKIP_IF_JUDGE_ERROR = True
REQUIRE_JUDGE_CONFIDENCE = True
MIN_JUDGE_CONF = 0.70

# Generation
MAX_NEW_TOKENS = 512
TEMPERATURE = 0.7
TOP_P = 0.9
GEN_ATTEMPTS = 3

SYSTEM_A = "You are a finance expert. Be concise, direct, and effective. Prefer short bullets. Avoid filler."
SYSTEM_B = "You are a finance expert. Be clear, professional, and helpful. Explain simply. Use a brief example if helpful."

# Judge (preference)
JUDGE_MODEL = "gpt-4.1-mini"
JUDGE_CONCURRENCY = 12
JUDGE_RETRIES = 3

# Fact-check gate (targeted)
FACTCHECK_MODEL = "gpt-4.1-mini"
FACTCHECK_RETRIES = 2
FACTCHECK_CONCURRENCY = 12
FACTCHECK_ENABLED = True
FACTCHECK_MIN_CONF = 0.70        # skip if fact-check confidence < this
FACTCHECK_TRIGGER_CONF = 0.80    # only fact-check if judge_conf >= this AND risk flagged

# Quality gates
MIN_CHARS = 180
MAX_SIMILARITY = 0.92
REQUIRE_END_PUNCT = True

# -----------------------------
# Required: questions list
# -----------------------------
# If you already defined questions in another cell, do NOT reassign here.
# questions = [...]

assert isinstance(questions, list) and len(questions) > 0, "questions must be a non-empty list[str]"

# -----------------------------
# Output helpers (create + streaming append)
# -----------------------------
def ensure_outfile(path: str) -> str:
    p = Path(path)
    p.parent.mkdir(parents=True, exist_ok=True)
    if not p.exists():
        p.write_text("", encoding="utf-8")
        print(f"[INIT] Created output file: {p.resolve()}")
    else:
        print(f"[INIT] Using existing output file: {p.resolve()}")
    return str(p)

OUT_FILE = ensure_outfile(OUT_FILE)

def append_jsonl(path: str, record: Dict[str, Any]) -> None:
    line = json.dumps(record, ensure_ascii=False)
    with open(path, "a", encoding="utf-8") as f:
        f.write(line + "\n")
        f.flush()
        os.fsync(f.fileno())

# -----------------------------
# Load FT model once
# -----------------------------
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=BASE_MODEL_ID,
    max_seq_length=4096,
    load_in_4bit=True,
)
model = PeftModel.from_pretrained(model, FT_ADAPTER_PATH)
model = FastLanguageModel.for_inference(model)

def _extract_assistant(text: str) -> str:
    marker = "<|start_header_id|>assistant<|end_header_id|>"
    if marker in text:
        text = text.split(marker, 1)[-1]
    return text.replace("<|eot_id|>", "").strip()

def _looks_truncated(ans: str) -> bool:
    if not ans:
        return True
    s = ans.strip()
    if REQUIRE_END_PUNCT and (s[-1] not in ".!?\"')]}"):
        last_line = s.splitlines()[-1].strip()
        if len(last_line) < 25 or (last_line[-1] not in ".!?\"')]}"):
            return True
    bad_endings = (" To", " And", " Or", " Because", " When", " Which", " With", " Such as", " For example", " e.g.")
    return any(s.endswith(x) for x in bad_endings)

def _simple_similarity(a: str, b: str) -> float:
    ta = set(re.findall(r"\w+", a.lower()))
    tb = set(re.findall(r"\w+", b.lower()))
    if not ta or not tb:
        return 1.0
    return len(ta & tb) / len(ta | tb)

@torch.no_grad()
def gen_answer(system_prompt: str, user_prompt: str) -> str:
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]
    input_ids = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt",
    ).to(model.device)

    out = model.generate(
        input_ids=input_ids,
        max_new_tokens=MAX_NEW_TOKENS,
        temperature=TEMPERATURE,
        top_p=TOP_P,
        do_sample=True,
        use_cache=True,
    )
    return _extract_assistant(tokenizer.decode(out[0], skip_special_tokens=False))

def generate_pair(question: str) -> Tuple[str, str]:
    last_a, last_b = "", ""
    for _ in range(GEN_ATTEMPTS):
        a = gen_answer(SYSTEM_A, question)
        b = gen_answer(SYSTEM_B, question)
        last_a, last_b = a, b

        if len(a) < MIN_CHARS or len(b) < MIN_CHARS:
            continue
        if _looks_truncated(a) or _looks_truncated(b):
            continue
        sim = _simple_similarity(a, b)
        if sim >= MAX_SIMILARITY:
            continue
        return a, b

    return last_a, last_b

# -----------------------------
# OpenAI client
# -----------------------------
client = AsyncOpenAI()

# -----------------------------
# Preference judge (async) + confidence + rationale
# -----------------------------
JUDGE_SYSTEM = (
    "You are a strict finance/investing expert advisor who helps both wealthy and low-income families. "
    "You prioritize factual correctness and sound reasoning over verbosity. "
    "Penalize math/logic errors, hallucinated claims, and unfinished/truncated answers. "
    "If both are similar, choose the one that is more correct and more actionable with fewer words."
)

async def judge(question: str, a: str, b: str) -> Dict[str, Any]:
    user = f"""
Question:
{question}

Answer A:
{a}

Answer B:
{b}

Choose the better answer by this rubric (in order):
1) Factual correctness / no hallucinations
2) Sound finance reasoning + correct caveats
3) Practical usefulness for both rich and low-income families (actionable, risk-aware)
4) Clarity and concision (prefer shorter if equally correct)

Return ONLY strict JSON:
{{
  "winner": "A" or "B",
  "confidence": 0.0-1.0,
  "rationale": "max 1 sentence"
}}
""".strip()

    last_err = None
    for attempt in range(1, JUDGE_RETRIES + 1):
        try:
            r = await client.chat.completions.create(
                model=JUDGE_MODEL,
                messages=[
                    {"role": "system", "content": JUDGE_SYSTEM},
                    {"role": "user", "content": user},
                ],
                temperature=0,
            )
            raw = (r.choices[0].message.content or "").strip()

            parsed = None
            try:
                parsed = json.loads(raw)
            except Exception:
                m = re.search(r'\{[\s\S]*?\}', raw)
                if m:
                    try:
                        parsed = json.loads(m.group(0))
                    except Exception:
                        parsed = None

            if parsed and parsed.get("winner") in ("A", "B"):
                return {
                    "winner": parsed["winner"],
                    "confidence": parsed.get("confidence", 0.0),
                    "rationale": parsed.get("rationale", ""),
                    "raw": raw,
                }

            last_err = f"Could not parse judge JSON. Raw head: {raw[:200]}"
        except Exception as e:
            last_err = str(e)

        await asyncio.sleep(0.5 * attempt)

    return {"winner": "A", "confidence": 0.0, "rationale": "", "raw": "", "error": last_err}

# -----------------------------
# Fact-check gate (async)
# Returns: {"action": "keep"|"flip"|"skip", "confidence": 0-1, "issues": [...], "raw": "..."}
# -----------------------------
FACTCHECK_SYSTEM = (
    "You are a meticulous finance expert and fact-checker. "
    "Your job is to prevent incorrect preferred answers from entering a DPO dataset. "
    "Be strict about math, definitions, causal claims, and unstable numeric claims. "
    "If both answers are flawed or too speculative, choose SKIP."
)

def _risk_flag(question: str, a: str, b: str, judge_rationale: str) -> bool:
    q = question.lower()
    ra = a.lower()
    rb = b.lower()
    rr = (judge_rationale or "").lower()

    # theory-heavy / easy-to-hallucinate topics
    risky_terms = [
        "risk-neutral", "stochastic discount factor", "sdf", "state price", "no-arbitrage",
        "capm", "consumption", "expectations hypothesis", "term structure", "duration", "convexity",
        "var", "expected shortfall", "cvar", "garch", "newey", "hansen-hodrick", "autocorrelation",
        "sharpe", "sortino", "beta", "alpha", "regression", "leakage", "overfitting", "backtest"
    ]
    if any(t in q for t in risky_terms):
        return True

    # numbers / ranges / % (unstable claims)
    if re.search(r"\b\d+(\.\d+)?\s*%|\b\d+(\.\d+)?\s*(bp|bps)\b", a) or re.search(r"\b\d+(\.\d+)?\s*%|\b\d+(\.\d+)?\s*(bp|bps)\b", b):
        return True

    # judge rationale mentions math/inaccuracy
    if "math" in rr or "incorrect" in rr or "inaccurate" in rr or "formula" in rr:
        return True

    return False

async def fact_check(question: str, a: str, b: str, winner: str) -> Dict[str, Any]:
    chosen = a if winner == "A" else b
    rejected = b if winner == "A" else a

    user = f"""
We are building DPO preference data for a finance expert assistant.

Question:
{question}

Chosen answer (currently preferred):
{chosen}

Rejected answer:
{rejected}

Task:
1) Check the CHOSEN answer for factual correctness and sound finance reasoning.
2) If CHOSEN is clearly better and correct, return action="keep".
3) If REJECTED is actually more correct/helpful, return action="flip".
4) If BOTH are flawed/speculative/contain math mistakes or unstable numeric claims, return action="skip".

Return ONLY strict JSON:
{{
  "action": "keep" or "flip" or "skip",
  "confidence": 0.0-1.0,
  "issues": ["short bullet points of concrete problems, if any (max 3)"]
}}
""".strip()

    last_err = None
    for attempt in range(1, FACTCHECK_RETRIES + 1):
        try:
            r = await client.chat.completions.create(
                model=FACTCHECK_MODEL,
                messages=[
                    {"role": "system", "content": FACTCHECK_SYSTEM},
                    {"role": "user", "content": user},
                ],
                temperature=0,
            )
            raw = (r.choices[0].message.content or "").strip()

            parsed = None
            try:
                parsed = json.loads(raw)
            except Exception:
                m = re.search(r'\{[\s\S]*?\}', raw)
                if m:
                    try:
                        parsed = json.loads(m.group(0))
                    except Exception:
                        parsed = None

            if parsed and parsed.get("action") in ("keep", "flip", "skip"):
                conf = parsed.get("confidence", 0.0)
                try:
                    conf = float(conf) if conf is not None else 0.0
                except Exception:
                    conf = 0.0
                return {
                    "action": parsed["action"],
                    "confidence": conf,
                    "issues": parsed.get("issues", []),
                    "raw": raw,
                }

            last_err = f"Could not parse fact-check JSON. Raw head: {raw[:200]}"
        except Exception as e:
            last_err = str(e)

        await asyncio.sleep(0.4 * attempt)

    return {"action": "skip", "confidence": 0.0, "issues": ["fact_check_parse_failed"], "raw": "", "error": last_err}

# -----------------------------
# Main loop (sequential gen; judge/fact-check async; streaming write)
# -----------------------------
async def run():
    total = len(questions)
    saved = 0
    skipped = 0
    flipped = 0
    factchecked = 0

    # dedup by prompt from existing file
    seen_prompts = set()
    with open(OUT_FILE, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            try:
                obj = json.loads(line)
                if "prompt" in obj:
                    seen_prompts.add(obj["prompt"])
            except Exception:
                continue

    print(f"[START] cwd={Path.cwd().resolve()}")
    print(f"[START] total_questions={total} already_in_file={len(seen_prompts)} out={Path(OUT_FILE).resolve()}")

    judge_sem = asyncio.Semaphore(JUDGE_CONCURRENCY)
    fact_sem = asyncio.Semaphore(FACTCHECK_CONCURRENCY)

    async def judge_with_sem(q, a, b):
        async with judge_sem:
            return await judge(q, a, b)

    async def fact_with_sem(q, a, b, winner):
        async with fact_sem:
            return await fact_check(q, a, b, winner)

    for i, q in enumerate(questions, 1):
        if q in seen_prompts:
            skipped += 1
            if DEBUG_EVERY:
                print(f"Q{i}/{total} – SKIP duplicate")
            continue

        # 1) Generate A/B sequentially (CUDA-safe)
        t0 = time.time()
        a, b = generate_pair(q)
        sim = _simple_similarity(a, b)

        reasons = []
        if len(a) < MIN_CHARS or len(b) < MIN_CHARS:
            reasons.append("too_short")
        if _looks_truncated(a) or _looks_truncated(b):
            reasons.append("truncated")
        if sim >= MAX_SIMILARITY:
            reasons.append(f"too_similar({sim:.2f})")

        if reasons:
            skipped += 1
            if DEBUG_EVERY:
                print(f"Q{i}/{total} – SKIP gen: {', '.join(reasons)}")
            continue

        # 2) Judge (I/O bound)
        j = await judge_with_sem(q, a, b)
        winner = j.get("winner", "A")
        conf = j.get("confidence", 0.0)
        judge_err = j.get("error", None)

        if SKIP_IF_JUDGE_ERROR and judge_err:
            skipped += 1
            if DEBUG_EVERY:
                print(f"Q{i}/{total} – SKIP judge_error: {judge_err}")
            continue

        try:
            conf_val = float(conf) if conf is not None else 0.0
        except Exception:
            conf_val = 0.0

        if REQUIRE_JUDGE_CONFIDENCE and conf_val < MIN_JUDGE_CONF:
            skipped += 1
            if DEBUG_EVERY:
                print(f"Q{i}/{total} – SKIP low_confidence: {conf_val:.2f}")
            continue

        # 3) Targeted fact-check (only if risk flagged)
        fc = None
        if FACTCHECK_ENABLED and conf_val >= FACTCHECK_TRIGGER_CONF and _risk_flag(q, a, b, j.get("rationale", "")):
            factchecked += 1
            fc = await fact_with_sem(q, a, b, winner)
            action = fc.get("action", "skip")
            fc_conf = float(fc.get("confidence", 0.0) or 0.0)

            if action == "skip" or fc_conf < FACTCHECK_MIN_CONF:
                skipped += 1
                if DEBUG_EVERY:
                    print(f"Q{i}/{total} – SKIP factcheck action={action} conf={fc_conf:.2f} issues={fc.get('issues')}")
                continue

            if action == "flip":
                winner = "B" if winner == "A" else "A"
                flipped += 1

        chosen = a if winner == "A" else b
        rejected = b if winner == "A" else a

        record = {
            "prompt": q,
            "chosen": chosen,
            "rejected": rejected,
            "meta": {
                "winner": winner,
                "confidence": conf_val,
                "rationale": j.get("rationale", ""),
                "system_A": SYSTEM_A,
                "system_B": SYSTEM_B,
                "judge_model": JUDGE_MODEL,
                "judge_raw": j.get("raw", ""),
                "judge_error": judge_err,
                "factchecked": bool(fc),
                "factcheck_action": (fc or {}).get("action", None),
                "factcheck_confidence": (fc or {}).get("confidence", None),
                "factcheck_issues": (fc or {}).get("issues", None),
                "factcheck_raw": (fc or {}).get("raw", None),
                "len_A": len(a),
                "len_B": len(b),
                "similarity": sim,
                "gen_seconds": round(time.time() - t0, 2),
            },
        }

        # 4) STREAM write immediately
        append_jsonl(OUT_FILE, record)
        seen_prompts.add(q)
        saved += 1

        if DEBUG_EVERY:
            extra = ""
            if fc:
                extra = f" fc={record['meta']['factcheck_action']} fc_conf={float(record['meta']['factcheck_confidence'] or 0):.2f}"
            print(
                f"Q{i}/{total} – {winner} saved={saved} skipped={skipped} flipped={flipped} "
                f"conf={conf_val:.2f} sim={sim:.2f}{extra}"
            )

    print(
        f"\n[DONE] saved={saved} skipped={skipped} flipped={flipped} "
        f"factchecked={factchecked} out={Path(OUT_FILE).resolve()}"
    )

await run()




[INIT] Using existing output file: /home/jovyan/work/MLE_in_Gen_AI-Course/class7/Homework_Kevin/DPO_preference_data.jsonl
==((====))==  Unsloth 2025.12.9: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    inference-ai GPU cuda. Num GPUs = 1. Max memory: 47.988 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 8.6. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
[START] cwd=/home/jovyan/work/MLE_in_Gen_AI-Course/class7/Homework_Kevin
[START] total_questions=200 already_in_file=8 out=/home/jovyan/work/MLE_in_Gen_AI-Course/class7/Homework_Kevin/DPO_preference_data.jsonl
Q1/200 – SKIP duplicate
Q2/200 – SKIP duplicate
Q3/200 – SKIP duplicate
Q4/200 – SKIP duplicate
Q5/200 – SKIP duplicate
Q6/200 – SKIP duplicate
Q7/200 – SKIP duplicate
Q8/200 – SKIP duplicate
Q9/200 – B save

In [8]:
# ✅ BEST full solution: Unsloth 4-bit base + your SFT LoRA + TRL DPO (uses DPOConfig)
# - Loads your JSONL preferences: {"prompt","chosen","rejected",...}
# - Policy = base + SFT adapter (trainable)
# - Ref    = base + SFT adapter (frozen)
# - Saves DPO adapter to: llama31-8b-finance-dpo-lora-kevin
#
# If you hit an import error for DPOConfig (older TRL), tell me your `pip show trl` version.

import os
import torch
from datasets import load_dataset
from peft import PeftModel
from trl import DPOTrainer, DPOConfig
from unsloth import FastLanguageModel

# -----------------------------
# Config
# -----------------------------
BASE_MODEL_ID      = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit"
SFT_ADAPTER_PATH   = "llama31-8b-finance-qlora-lora"
DATA_FILE          = "DPO_preference_data.jsonl"   # <- make sure this is .jsonl
OUT_DIR            = "llama31-8b-finance-dpo-lora-kevin"

MAX_SEQ_LEN        = 4096
MAX_LENGTH         = 2048
MAX_PROMPT_LENGTH  = 1024
BETA               = 0.1

# Training
PER_DEVICE_BS      = 1
GRAD_ACCUM         = 8
LR                 = 5e-6
EPOCHS             = 1
LOGGING_STEPS      = 10
SAVE_STEPS         = 200
WARMUP_RATIO       = 0.03
JIT_COMPILE        = False  # set True only if you know it works in your env

# -----------------------------
# Safety checks
# -----------------------------
assert os.path.exists(DATA_FILE), f"Missing data file: {DATA_FILE}"
assert os.path.exists(SFT_ADAPTER_PATH), f"Missing adapter dir: {SFT_ADAPTER_PATH}"

# -----------------------------
# Load dataset
# -----------------------------
dataset = load_dataset("json", data_files=DATA_FILE)["train"]
required_cols = {"prompt", "chosen", "rejected"}
missing = required_cols - set(dataset.column_names)
assert not missing, f"Dataset missing required columns: {missing}"

# (Optional) light cleanup if you accidentally stored empty lines
dataset = dataset.filter(lambda x: isinstance(x["prompt"], str) and len(x["prompt"].strip()) > 0)

# -----------------------------
# Load tokenizer via Unsloth
# -----------------------------
_, tokenizer = FastLanguageModel.from_pretrained(
    model_name=BASE_MODEL_ID,
    max_seq_length=MAX_SEQ_LEN,
    load_in_4bit=True,
)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# -----------------------------
# Policy model (trainable)
# -----------------------------
policy, _ = FastLanguageModel.from_pretrained(
    model_name=BASE_MODEL_ID,
    max_seq_length=MAX_SEQ_LEN,
    load_in_4bit=True,
)
policy = PeftModel.from_pretrained(policy, SFT_ADAPTER_PATH, is_trainable=True)

# -----------------------------
# Reference model (frozen)
# -----------------------------
ref, _ = FastLanguageModel.from_pretrained(
    model_name=BASE_MODEL_ID,
    max_seq_length=MAX_SEQ_LEN,
    load_in_4bit=True,
)
ref = PeftModel.from_pretrained(ref, SFT_ADAPTER_PATH, is_trainable=False)
ref.eval()
for p in ref.parameters():
    p.requires_grad = False

# -----------------------------
# DPO config (TRL expects this, not TrainingArguments)
# -----------------------------
dpo_args = DPOConfig(
    output_dir=OUT_DIR,
    per_device_train_batch_size=PER_DEVICE_BS,
    gradient_accumulation_steps=GRAD_ACCUM,
    learning_rate=LR,
    num_train_epochs=EPOCHS,
    logging_steps=LOGGING_STEPS,
    save_steps=SAVE_STEPS,
    report_to="none",

    # precision
    bf16=torch.cuda.is_bf16_supported(),
    fp16=not torch.cuda.is_bf16_supported(),

    # optimizer good for 4-bit + LoRA
    optim="paged_adamw_8bit",

    lr_scheduler_type="cosine",
    warmup_ratio=WARMUP_RATIO,

    # DPO-specific
    beta=BETA,
    max_length=MAX_LENGTH,
    max_prompt_length=MAX_PROMPT_LENGTH,
)

# -----------------------------
# Trainer
# -----------------------------
trainer = DPOTrainer(
    model=policy,
    ref_model=ref,
    args=dpo_args,
    train_dataset=dataset,
    tokenizer=tokenizer,
)

# (Optional) compile
if JIT_COMPILE and hasattr(torch, "compile"):
    trainer.model = torch.compile(trainer.model)

# -----------------------------
# Train + save
# -----------------------------
trainer.train()
trainer.save_model(OUT_DIR)
tokenizer.save_pretrained(OUT_DIR)

print("✅ DPO-aligned model saved to:", OUT_DIR)



Filter: 100%|██████████| 190/190 [00:00<00:00, 5435.26 examples/s]


==((====))==  Unsloth 2025.12.9: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    inference-ai GPU cuda. Num GPUs = 1. Max memory: 47.988 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 8.6. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
==((====))==  Unsloth 2025.12.9: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    inference-ai GPU cuda. Num GPUs = 1. Max memory: 47.988 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 8.6. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
==((====))==  Unsloth 2025.12.9: Fast Llama patching. Transformers: 4.57.3.
  

Extracting prompt in train dataset (num_proc=64): 100%|██████████| 190/190 [00:08<00:00, 22.74 examples/s] 
Applying chat template to train dataset (num_proc=64): 100%|██████████| 190/190 [00:25<00:00,  7.31 examples/s]
Tokenizing train dataset (num_proc=64): 100%|██████████| 190/190 [00:26<00:00,  7.25 examples/s]
The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 190 | Num Epochs = 1 | Total steps = 24
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 8 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,rewards / chosen,rewards / rejected,rewards / accuracies,rewards / margins,logps / chosen,logps / rejected,logits / chosen,logits / rejected,eval_logits / chosen,eval_logits / rejected,nll_loss
10,0.678100,0.104784,0.067752,0.587500,0.037032,-172.858185,-158.079453,-0.568802,-0.586924,0,0,0
20,0.652200,0.268530,0.165223,0.637500,0.103307,-165.902496,-158.191895,-0.564892,-0.577850,No Log,No Log,No Log


✅ DPO-aligned model saved to: llama31-8b-finance-dpo-lora-kevin


In [ ]:
# Finetuning eval (3-way):
# Compare:
#  1) ORIGINAL base: unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit
#  2) QLoRA adapter: llama31-8b-finance-qlora-lora
#  3) DPO LoRA adapter: llama31-8b-finance-dpo-lora-kevin
#
# Outputs:
#  - per-question answers for all 3
#  - pairwise judge results (Base-as-judge): Original vs QLoRA, Original vs DPO, QLoRA vs DPO
#  - simple aggregate win counts + average scores
#  - saves JSON: eval_3way_results.json
#
# Deps:
# pip install -U unsloth transformers peft accelerate bitsandbytes torch

import time, json
import torch
from unsloth import FastLanguageModel
from peft import PeftModel

# -----------------------------
# Config
# -----------------------------
BASE_MODEL_ID = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit"

ADAPTERS = {
    "qlora": "llama31-8b-finance-qlora-lora",        # local folder OR HF repo id
    "dpo":   "llama31-8b-finance-dpo-lora-kevin",    # local folder OR HF repo id
}

MAX_NEW_TOKENS = 384
TEMPERATURE = 0.2
TOP_P = 0.9
SEED = 42
MAX_SEQ_LEN = 4096

torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

# -----------------------------
# Test questions
# -----------------------------
test_questions = [
    "Explain look-ahead bias in backtesting and how to avoid it.",
    "What is survivorship bias and why does it matter in equity datasets?",
    "Compare VaR and Expected Shortfall conceptually. When is ES preferred?",
    "Why might a model have high AUC but poor calibration?",
    "Why can a high Sharpe ratio be misleading in small samples?",
]

# -----------------------------
# Helpers
# -----------------------------
def load_base_model():
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=BASE_MODEL_ID,
        max_seq_length=MAX_SEQ_LEN,
        load_in_4bit=True,
        dtype=None,
    )
    model = FastLanguageModel.for_inference(model)
    return model, tokenizer

def load_model_with_adapter(adapter_path: str):
    """
    Load a fresh base model instance, attach a PEFT LoRA adapter, then set for inference.
    """
    model, _ = FastLanguageModel.from_pretrained(
        model_name=BASE_MODEL_ID,
        max_seq_length=MAX_SEQ_LEN,
        load_in_4bit=True,
        dtype=None,
    )
    model = PeftModel.from_pretrained(model, adapter_path)
    model = FastLanguageModel.for_inference(model)
    return model

def format_chat(tokenizer, user_text: str):
    messages = [
        {"role": "system", "content": "You are a helpful finance and investing assistant. Be accurate, concise, and practical."},
        {"role": "user", "content": user_text},
    ]
    input_ids = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt",
    )
    return input_ids

@torch.no_grad()
def generate_answer(model, tokenizer, question: str):
    input_ids = format_chat(tokenizer, question).to(model.device)
    attention_mask = torch.ones_like(input_ids, device=model.device)

    gen = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=MAX_NEW_TOKENS,
        temperature=TEMPERATURE,
        do_sample=(TEMPERATURE > 0),
        top_p=TOP_P,
        use_cache=True,
    )

    text = tokenizer.decode(gen[0], skip_special_tokens=False)

    # Best-effort extraction of assistant content
    for marker in ["<|assistant|>", "assistant\n", "### Assistant:"]:
        if marker in text:
            return text.split(marker, 1)[-1].strip()
    return text.strip()

def judge_pair_with_base(judge_model, tokenizer, question: str, ans_A: str, ans_B: str, label_A="A", label_B="B"):
    """
    Base-model-as-judge. Returns a JSON string (best-effort).
    """
    judge_prompt = f"""
You are grading two answers to the same finance/investing question.

Question:
{question}

Answer {label_A}:
{ans_A}

Answer {label_B}:
{ans_B}

Rubric (most important first):
1) Factual correctness (no hallucinations)
2) Finance-specific insight / domain depth
3) Practical usefulness (how-to, pitfalls, examples)
4) Clarity and concision

Return STRICT JSON only:
{{
  "winner": "{label_A}" or "{label_B}" or "tie",
  "score_{label_A}": 1-10,
  "score_{label_B}": 1-10,
  "rationale": "1-3 sentences"
}}
""".strip()

    input_ids = format_chat(tokenizer, judge_prompt).to(judge_model.device)
    attention_mask = torch.ones_like(input_ids, device=judge_model.device)

    gen = judge_model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        temperature=0.0,
        do_sample=False,
        use_cache=True,
    )

    out = tokenizer.decode(gen[0], skip_special_tokens=False)

    # best-effort extract JSON
    if "{" in out and "}" in out:
        out = out[out.find("{"): out.rfind("}") + 1]
    return out.strip()

def safe_parse_json(s: str):
    try:
        return json.loads(s)
    except Exception:
        return None

# -----------------------------
# Load models
# -----------------------------
print("Loading ORIGINAL base (also used as judge)...")
base_model, tokenizer = load_base_model()

print("Loading QLoRA adapter model...")
qlora_model = load_model_with_adapter(ADAPTERS["qlora"])

print("Loading DPO adapter model...")
dpo_model = load_model_with_adapter(ADAPTERS["dpo"])

models = {
    "original": base_model,
    "qlora": qlora_model,
    "dpo": dpo_model,
}

# -----------------------------
# Run 3-way evaluation
# -----------------------------
results = []
start = time.time()

# aggregate stats
win_counts = {
    "original": 0,
    "qlora": 0,
    "dpo": 0,
    "tie": 0,
}
score_sums = {"original": 0.0, "qlora": 0.0, "dpo": 0.0}
score_counts = {"original": 0, "qlora": 0, "dpo": 0}

pairings = [
    ("original", "qlora"),
    ("original", "dpo"),
    ("qlora", "dpo"),
]

for i, q in enumerate(test_questions, 1):
    print(f"\n{'='*100}\nQ{i}: {q}\n{'='*100}")

    answers = {}
    for name, m in models.items():
        ans = generate_answer(m, tokenizer, q)
        answers[name] = ans
        print(f"\n--- {name.upper()} ANSWER --------------------------------------------------")
        print(ans)

    judges = {}
    for a, b in pairings:
        # Keep labels consistent so parsing is easy
        judge_json = judge_pair_with_base(
            judge_model=base_model,
            tokenizer=tokenizer,
            question=q,
            ans_A=answers[a],
            ans_B=answers[b],
            label_A=a,
            label_B=b,
        )
        judges[f"{a}_vs_{b}"] = judge_json
        print(f"\n--- JUDGE: {a} vs {b} (JSON) ----------------------------------------------")
        print(judge_json)

        parsed = safe_parse_json(judge_json)
        if parsed:
            winner = parsed.get("winner", "tie")
            if winner in win_counts:
                win_counts[winner] += 1
            else:
                win_counts["tie"] += 1

            # accumulate scores if present
            for name in [a, b]:
                k = f"score_{name}"
                if k in parsed:
                    try:
                        score_sums[name] += float(parsed[k])
                        score_counts[name] += 1
                    except Exception:
                        pass

    results.append({
        "question": q,
        "answers": answers,
        "judges": judges,
    })

elapsed = time.time() - start

# -----------------------------
# Summary
# -----------------------------
avg_scores = {}
for name in ["original", "qlora", "dpo"]:
    avg_scores[name] = (score_sums[name] / score_counts[name]) if score_counts[name] else None

print(f"\n\n{'='*100}")
print("SUMMARY")
print(f"Ran {len(test_questions)} questions with {len(pairings)} pairwise judgments each "
      f"({len(test_questions)*len(pairings)} total judge calls) in {elapsed:.1f}s.\n")

print("Win counts (pairwise):")
for k, v in win_counts.items():
    print(f"  {k:>8}: {v}")

print("\nAverage judge scores (if parsable):")
for k, v in avg_scores.items():
    print(f"  {k:>8}: {v if v is not None else 'n/a'}")

# -----------------------------
# Save
# -----------------------------
out = {
    "base_model_id": BASE_MODEL_ID,
    "adapters": ADAPTERS,
    "gen_config": {
        "max_new_tokens": MAX_NEW_TOKENS,
        "temperature": TEMPERATURE,
        "top_p": TOP_P,
        "seed": SEED,
        "max_seq_len": MAX_SEQ_LEN,
    },
    "pairings": pairings,
    "results": results,
    "summary": {
        "elapsed_sec": elapsed,
        "win_counts": win_counts,
        "avg_scores": avg_scores,
        "score_counts": score_counts,
    },
}

with open("eval_results_alignment_ft_vs_base_vs_dpo.json", "w") as f:
    json.dump(out, f, indent=2)

print("\nSaved: eval_results_alignment_ft_vs_base_vs_dpo.json")


Loading ORIGINAL base (also used as judge)...
==((====))==  Unsloth 2025.12.9: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    inference-ai GPU cuda. Num GPUs = 1. Max memory: 47.988 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 8.6. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Loading QLoRA adapter model...
==((====))==  Unsloth 2025.12.9: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    inference-ai GPU cuda. Num GPUs = 1. Max memory: 47.988 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 8.6. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
L

In [ ]:
# 3-way Finetuning eval (OOM-safe, single-model, adapter swapping)
# Compare:
#  1) ORIGINAL base: unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit
#  2) QLoRA adapter: llama31-8b-finance-qlora-lora
#  3) DPO LoRA adapter: llama31-8b-finance-dpo-lora-kevin
#
# Judge: GPT-4.1-mini (recommended). No extra local judge model loaded.
#
# Output:
#  - per-question answers for all 3
#  - pairwise judge results: original_vs_qlora, original_vs_dpo, qlora_vs_dpo
#  - aggregates: win counts + avg scores
#  - saves: eval_3way_results.json
#
# pip install -U unsloth transformers peft accelerate bitsandbytes torch openai python-dotenv

import os, json, re, time, math, asyncio
from typing import Dict, Any, Optional, Tuple

import torch
from unsloth import FastLanguageModel
from peft import PeftModel
from dotenv import load_dotenv
from openai import AsyncOpenAI

load_dotenv()

# -----------------------------
# Config
# -----------------------------
BASE_MODEL_ID = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit"

ADAPTERS = {
    "qlora": "llama31-8b-finance-qlora-lora",        # local folder OR HF repo id
    "dpo":   "llama31-8b-finance-dpo-lora-kevin",    # local folder OR HF repo id
}

OUT_JSON = "eval_results_alignment_ft_vs_base_vs_dpo.json"

MAX_SEQ_LEN = 4096
MAX_NEW_TOKENS_ANS = 384
TEMPERATURE = 0.2
TOP_P = 0.9
SEED = 42

JUDGE_MODEL = "gpt-4.1-mini"
JUDGE_MAX_TOKENS = 220
JUDGE_CONCURRENCY = 8
JUDGE_RETRIES = 3

SYSTEM_ANSWER = "You are a helpful finance and investing assistant. Be accurate, concise, and practical."
JUDGE_SYSTEM = (
    "You are a strict evaluator for finance/investing answers. "
    "Prioritize: (1) factual correctness, (2) finance depth, (3) practical usefulness, (4) clarity. "
    "Return ONLY valid JSON, no extra text."
)

test_questions = [
    "Explain look-ahead bias in backtesting and how to avoid it.",
    "What is survivorship bias and why does it matter in equity datasets?",
    "Compare VaR and Expected Shortfall conceptually. When is ES preferred?",
    "Why might a model have high AUC but poor calibration?",
    "Why can a high Sharpe ratio be misleading in small samples?",
]

torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

# -----------------------------
# Model load (ONE TIME)
# -----------------------------
print("Loading base model once...")
base_model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=BASE_MODEL_ID,
    max_seq_length=MAX_SEQ_LEN,
    load_in_4bit=True,
    dtype=None,
)
base_model = FastLanguageModel.for_inference(base_model)

# Turn base into a PEFT-wrapped model once so we can load adapters into it
# (this avoids loading multiple base copies)
model = PeftModel.from_pretrained(base_model, ADAPTERS["qlora"])  # temporary
model = FastLanguageModel.for_inference(model)
model.set_adapter("default")  # adapter name in PEFT is often "default"

# Load second adapter into same model under a different name
# If load_adapter isn't available in your peft version, we'll fallback to loading via from_pretrained into the SAME wrapper.
if hasattr(model, "load_adapter"):
    model.load_adapter(ADAPTERS["dpo"], adapter_name="dpo")
else:
    # Fallback: attach second adapter using peft's loading API
    # This is a safe, common workaround if your peft is older.
    from peft import PeftConfig
    # This will still reuse the same base weights; adapter weights are added.
    model = PeftModel.from_pretrained(model, ADAPTERS["dpo"], adapter_name="dpo")
    model = FastLanguageModel.for_inference(model)

# Rename the first adapter to "qlora" if it's still "default"
# (Some PEFT versions keep it as "default". We'll map both cases.)
available = list(getattr(model, "peft_config", {}).keys())
if "qlora" not in available:
    if "default" in available:
        qlora_adapter_name = "default"
    else:
        # take first adapter as qlora
        qlora_adapter_name = available[0] if available else "default"
else:
    qlora_adapter_name = "qlora"

dpo_adapter_name = "dpo"

print("Adapters available:", list(getattr(model, "peft_config", {}).keys()))
print("Using qlora adapter name:", qlora_adapter_name)
print("Using dpo adapter name:", dpo_adapter_name)

# -----------------------------
# Generation utilities
# -----------------------------
def _chat_input_ids(system: str, user: str) -> torch.Tensor:
    messages = [{"role": "system", "content": system}, {"role": "user", "content": user}]
    return tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt",
    )

@torch.no_grad()
def generate_answer_variant(variant: str, question: str) -> str:
    """
    variant: "original" | "qlora" | "dpo"
    Uses the SAME model in memory; swaps adapters.
    """
    # switch adapter
    if variant == "original":
        # disable adapters
        if hasattr(model, "disable_adapter"):
            with model.disable_adapter():
                return _generate(question)
        else:
            # older PEFT: set_adapter(None) may not exist; best-effort by using base_model directly
            return _generate(question, use_base_only=True)

    elif variant == "qlora":
        model.set_adapter(qlora_adapter_name)
        return _generate(question)

    elif variant == "dpo":
        model.set_adapter(dpo_adapter_name)
        return _generate(question)

    else:
        raise ValueError(f"Unknown variant: {variant}")

@torch.no_grad()
def _generate(question: str, use_base_only: bool = False) -> str:
    m = base_model if use_base_only else model
    input_ids = _chat_input_ids(SYSTEM_ANSWER, question).to(m.device)
    in_len = input_ids.shape[-1]

    out = m.generate(
        input_ids=input_ids,
        attention_mask=torch.ones_like(input_ids, device=m.device),
        max_new_tokens=MAX_NEW_TOKENS_ANS,
        temperature=TEMPERATURE,
        do_sample=(TEMPERATURE > 0),
        top_p=TOP_P,
        use_cache=True,
    )
    # decode only new tokens
    new_tokens = out[0, in_len:]
    text = tokenizer.decode(new_tokens, skip_special_tokens=True)
    return text.strip()

# -----------------------------
# Judge (GPT) utilities
# -----------------------------
client = AsyncOpenAI()

def _extract_json(text: str) -> Optional[Dict[str, Any]]:
    if not text:
        return None
    text = text.strip()
    try:
        return json.loads(text)
    except Exception:
        pass
    m = re.search(r"\{[\s\S]*?\}", text)
    if m:
        s = m.group(0)
        s = re.sub(r",\s*([}\]])", r"\1", s)
        try:
            return json.loads(s)
        except Exception:
            return None
    return None

def _coerce_winner(w: Any, a: str, b: str) -> str:
    if w is None:
        return "tie"
    s = str(w).strip().lower()
    if s == "tie":
        return "tie"
    if s in ("a", a.lower()):
        return a
    if s in ("b", b.lower()):
        return b
    return "tie"

def _coerce_score(x: Any) -> Optional[float]:
    try:
        v = float(x)
        if math.isfinite(v):
            return max(1.0, min(10.0, v))
        return None
    except Exception:
        return None

async def judge_pair(question: str, ansA: str, ansB: str, labelA: str, labelB: str) -> Dict[str, Any]:
    prompt = f"""
Grade two answers to the same finance/investing question.

Question:
{question}

Answer {labelA}:
{ansA}

Answer {labelB}:
{ansB}

Return ONLY strict JSON:
{{
  "winner": "{labelA}" or "{labelB}" or "tie",
  "score_{labelA}": 1-10,
  "score_{labelB}": 1-10,
  "rationale": "1-3 sentences"
}}
""".strip()

    last_err = None
    for attempt in range(1, JUDGE_RETRIES + 1):
        try:
            r = await client.chat.completions.create(
                model=JUDGE_MODEL,
                messages=[
                    {"role": "system", "content": JUDGE_SYSTEM},
                    {"role": "user", "content": prompt},
                ],
                temperature=0,
                max_tokens=JUDGE_MAX_TOKENS,
            )
            raw = (r.choices[0].message.content or "").strip()
            obj = _extract_json(raw)
            if isinstance(obj, dict):
                winner = _coerce_winner(obj.get("winner"), labelA, labelB)
                sA = _coerce_score(obj.get(f"score_{labelA}"))
                sB = _coerce_score(obj.get(f"score_{labelB}"))
                rationale = str(obj.get("rationale", "")).strip()
                return {
                    "final": {"winner": winner, f"score_{labelA}": sA, f"score_{labelB}": sB, "rationale": rationale},
                    "raw": raw,
                    "parsed_debug": obj,
                }
            last_err = f"Unparsable judge JSON (head): {raw[:200]}"
        except Exception as e:
            last_err = str(e)
        await asyncio.sleep(0.4 * attempt)

    return {
        "final": {"winner": "tie", f"score_{labelA}": None, f"score_{labelB}": None, "rationale": ""},
        "raw": "",
        "parsed_debug": None,
        "error": last_err,
    }

# -----------------------------
# Main eval
# -----------------------------
async def main():
    pairings = [("original", "qlora"), ("original", "dpo"), ("qlora", "dpo")]

    results = []
    win_counts = {"original": 0, "qlora": 0, "dpo": 0, "tie": 0}
    score_sums  = {"original": 0.0, "qlora": 0.0, "dpo": 0.0}
    score_counts= {"original": 0, "qlora": 0, "dpo": 0}

    start = time.time()
    sem = asyncio.Semaphore(JUDGE_CONCURRENCY)

    async def judge_with_sem(q, a, b, la, lb):
        async with sem:
            return await judge_pair(q, a, b, la, lb)

    for i, q in enumerate(test_questions, 1):
        print(f"\n{'='*100}\nQ{i}: {q}\n{'='*100}")

        # Generate 3 answers (sequential, CUDA-safe)
        answers = {}
        for v in ["original", "qlora", "dpo"]:
            ans = generate_answer_variant(v, q)
            answers[v] = ans
            print(f"\n--- {v.upper()} ANSWER --------------------------------------------------")
            print(ans)

        # Judge pairwise (concurrent across 3 pairings)
        judge_tasks = []
        for a, b in pairings:
            judge_tasks.append(judge_with_sem(q, answers[a], answers[b], a, b))

        judge_outs = await asyncio.gather(*judge_tasks)

        judges = {}
        for (a, b), jr in zip(pairings, judge_outs):
            key = f"{a}_vs_{b}"
            judges[key] = jr
            final = jr["final"]
            print(f"\n--- JUDGE: {a} vs {b} ----------------------------------------------------")
            print(json.dumps(final, indent=2))

            # aggregate
            winner = final.get("winner", "tie")
            win_counts[winner] = win_counts.get(winner, 0) + 1

            for name in (a, b):
                sv = final.get(f"score_{name}", None)
                if sv is not None:
                    score_sums[name] += float(sv)
                    score_counts[name] += 1

        results.append({"question": q, "answers": answers, "judges": judges})

        # free some KV cache between questions
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

    elapsed = time.time() - start
    avg_scores = {k: (score_sums[k] / score_counts[k] if score_counts[k] else None) for k in score_sums}

    print(f"\n\n{'='*100}\nSUMMARY")
    print(f"elapsed_sec={elapsed:.1f}")
    print("Win counts:")
    for k, v in win_counts.items():
        print(f"  {k:>8}: {v}")
    print("Avg scores:")
    for k, v in avg_scores.items():
        print(f"  {k:>8}: {v if v is not None else 'n/a'}")

    out = {
        "base_model_id": BASE_MODEL_ID,
        "adapters": ADAPTERS,
        "gen_config": {
            "max_new_tokens_answer": MAX_NEW_TOKENS_ANS,
            "temperature": TEMPERATURE,
            "top_p": TOP_P,
            "seed": SEED,
            "max_seq_len": MAX_SEQ_LEN,
        },
        "judge": {"model": JUDGE_MODEL},
        "pairings": pairings,
        "results": results,
        "summary": {
            "elapsed_sec": elapsed,
            "win_counts": win_counts,
            "avg_scores": avg_scores,
            "score_counts": score_counts,
        },
    }

    with open(OUT_JSON, "w", encoding="utf-8") as f:
        json.dump(out, f, indent=2, ensure_ascii=False)

    print(f"\nSaved: {OUT_JSON}")

await main()


Loading base model once...
==((====))==  Unsloth 2025.12.9: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    inference-ai GPU cuda. Num GPUs = 1. Max memory: 47.988 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 8.6. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Adapters available: ['default', 'dpo']
Using qlora adapter name: default
Using dpo adapter name: dpo

Q1: Explain look-ahead bias in backtesting and how to avoid it.

--- ORIGINAL ANSWER --------------------------------------------------
**What is Look-Ahead Bias in Backtesting?**

Look-ahead bias, also known as data snooping or overfitting, is a common issue in backtesting that occurs when a model or strategy is optimized using data that is not available at the time of the investment decision. This means t